In [ ]:
import numpy as np
from functools import lru_cache

# 暖冷房負荷ファイルの読み込み
from section3_1_file import \
    get_filename, \
    get_L_dash_H_R_TSl_Qj_muH_j_k_d_t_i, \
    get_L_dash_CS_R_NVl_Qj_muH_j_k_d_t_i, \
    get_L_dash_CL_R_NVl_Qj_muH_j_k_d_t_i

# 床下空間を経由して外気を導入する方式
import section3_1_d as uf

# 空気集熱式太陽熱利用設備による負荷削減量
import section9_3 as ass

from section11_1 import load_outdoor, get_Theta_ex


# 第三章　暖冷房負荷と外皮性能 / Chapter 3 Heating and cooling load and shell performance

## 第一節　全般 / Section 1 General

1．適用範囲～4．記号及び単位は省略

1. Scope of application ~ 4. Symbols and units are omitted

### 5．暖冷房負荷の補正 / Correction of heating and cooling load

$$\large
L_{H,d,t,i} = L'_{H,d,t,i} \times f_{R,Evp,i} \times f_{R,Eqp,i} \times f_{TD,i}  
$$

$L_{H,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の暖房負荷(MJ/h) / Heating load of heating / cooling section $ i $ at time $ t $ with date $ d $  
$L'_{H,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の負荷補正前の暖房負荷(MJ/h) / Heating load before load correction of heating / cooling section $ i $ at time $ t $ of date $ d $  
$f_{R,Evp,i}$：暖冷房区画$i$における外皮等の表面温度による放射温度を考慮した負荷補正係数 / Load correction factor considering radiation temperature due to surface temperature of hull etc. in heating / cooling section $ i $  
$f_{R,Eqp,i}$：暖冷房区画$i$における暖房設備の方式による放射温度を考慮した負荷補正係数 / Load correction factor considering the radiation temperature by the method of heating equipment in heating and cooling division $ i $  
$f_{TD,i}$：暖冷房区画$i$における上下温度分布を考慮した負荷補正係数 / Load correction factor considering temperature distribution in heating / cooling section $ i $  

In [ ]:
# 暖冷房区画i݅の１時間当たりの暖房負荷 (1)
@lru_cache()
def get_L_H_d_t_i(region, A_A, A_MR, A_OR, mode, TS, Q, mu_H, hex, etr_dash_t, r_A_ufvnt, underfloor_insulation,
                   R_l_i, floorheating, hotwater_use=None, supply_target=None, sol_region=None, P_alpha=None, P_beta=None,
                       A_col=None, V_fan_P0=None, m_fan_test=None, d0=None, d1=None, r_A_ufvnt_ass=None, ufv_insulation=None):
    L_H_d_t_i = np.zeros((12, 24 * 365))

    outdoor = load_outdoor()
    Theta_ex_d_t = get_Theta_ex(region, outdoor)

    # 負荷補正前の暖房負荷
    L_dash_H_d_t_i, L_dash_H_R_d_t_i = get_L_dash_H_d_t_i(
        region=region,
        A_A=A_A,
        A_MR=A_MR,
        A_OR=A_OR,
        mode=mode,
        TS=TS,
        Q=Q,
        mu_H=mu_H,
        hex=hex,
        etr_dash_t=etr_dash_t,
        r_A_ufvnt=r_A_ufvnt,
        underfloor_insulation=underfloor_insulation,
        Theta_ex_d_t=Theta_ex_d_t,
        hotwater_use=hotwater_use,
        supply_target=supply_target,
        sol_region=sol_region,
        P_alpha=P_alpha,
        P_beta=P_beta,
        A_col=A_col,
        V_fan_P0=V_fan_P0,
        m_fan_test=m_fan_test,
        d0=d0,
        d1=d1,
        r_A_ufvnt_ass=r_A_ufvnt_ass,
        ufv_insulation=ufv_insulation
    )

    # ----- 負荷補正の実施 -----
    for i in range(1, 13):
        if mode[i - 1] is not None:
            # 外皮等の表面温度による放射温度を考慮した負荷補正係数
            f_R_Evp_i = get_f_R_Evp_i(region=region, mode=mode[i - 1], Q=Q)

            # 暖房設備の方式による放射温度を考慮した負荷補正係数
            f_R_Eqp_i = get_f_R_Eqp_i(floorheating=floorheating[i - 1], mode=mode[i - 1], R_l_i=R_l_i[i - 1])

            # 上下温度分布を考慮した負荷補正係数
            f_TD_i = get_f_TD_i(floorheating=floorheating[i - 1], region=region, mode=mode[i - 1], Q=Q)

            L_H_d_t_i[i - 1, :] = L_dash_H_d_t_i[i - 1, :] * f_R_Evp_i * f_R_Eqp_i * f_TD_i  # (1)
        else:
            L_H_d_t_i[i - 1, :] = L_dash_H_d_t_i[i - 1, :]  # (1)

    return L_H_d_t_i, L_dash_H_R_d_t_i


$$\large
f_{R,Evp,i} = a_{R,Evp,i} \times Q + b_{R,Evp,i} 
$$

$Q$：熱損失係数(W/(m<sup>2</sup>K)) / Heat loss coefficient  

<div style="text-align: center;font-weight: bold;">表　係数$a_{R,Evp,i}$及び$b_{R,Evp,i}$<br>Table: Coefficients $ a_ {R, Evp, i} $ and $ b_ {R, Evp, i} $</div>

<table>
<thead>
<tr>
<th rowspan="4">地域の区分<br>Segmentation of area</th>
<th colspan="6"><center>暖房方式及び運転方法の区分<br>Classification of heating method and operation method</center></th>
</tr>
<tr>
<th colspan="2">（い）<br>(I)</th>
<th colspan="2">（ろ）<br>(RO)</th>
<th colspan="2">（は）<br>(HA)</th>
</tr>
<tr>
<th colspan="2">「住戸全体を連続的に<br>暖房する方式」<br>"Method to heat the entire dwelling unit continuously" </th>
<th colspan="2">「居室のみを暖房する方式」<br>かつ「連続運転」の場合 <br>In the case of "method to heat only the living room" and "continuous operation"</th>
<th colspan="2">「居室のみを暖房する方式」<br>かつ「間歇運転」の場合<br>In the case of "method to heat only the living room" and "intermittent operation"</th>
</tr>
<tr>
<th>$a_{R,Evp,i}$</th>
<th>$b_{R,Evp,i}$</th>
<th>$a_{R,Evp,i}$</th>
<th>$b_{R,Evp,i}$</th>
<th>$a_{R,Evp,i}$</th>
<th>$b_{R,Evp,i}$</th>
</tr>
</thead>

<tbody>
<tr>
<th>1</th>
<td>0.031</td>
<td>0.971</td>
<td>0.041</td>
<td>0.975</td>
<td>0.059</td>
<td>1.038</td>
</tr>
<tr>
<th>2</th>
<td>0.032</td>
<td>0.966</td>
<td>0.043</td>
<td>0.970</td>
<td>0.060</td>
<td>1.034</td>
</tr>
<tr>
<th>3</th>
<td>0.030</td>
<td>0.963</td>
<td>0.039</td>
<td>0.970</td>
<td>0.050</td>
<td>1.049</td>
</tr>
<tr>
<th>4</th>
<td>0.027</td>
<td>0.972</td>
<td>0.033</td>
<td>0.985</td>
<td>0.040</td>
<td>1.081</td>
</tr>
<tr>
<th>5</th>
<td>0.028</td>
<td>0.966</td>
<td>0.034</td>
<td>0.981</td>
<td>0.038</td>
<td>1.092</td>
</tr>
<tr>
<th>6</th>
<td>0.029</td>
<td>0.961</td>
<td>0.035</td>
<td>0.974</td>
<td>0.039</td>
<td>1.090</td>
</tr>
<tr>
<th>7</th>
<td>0.020</td>
<td>0.921</td>
<td>0.024</td>
<td>0.937</td>
<td>0.021</td>
<td>1.094</td>
</tr>
</tbody>
</table>


In [ ]:
# 外皮等の表面温度による放射温度を考慮した負荷補正係数 (2)
def get_f_R_Evp_i(region, mode, Q):
    # 表 3 式(2)における係数
    table_3 = [
        (0.031, 0.971, 0.041, 0.975, 0.059, 1.038),
        (0.032, 0.966, 0.043, 0.970, 0.060, 1.034),
        (0.030, 0.963, 0.039, 0.970, 0.050, 1.049),
        (0.027, 0.972, 0.033, 0.985, 0.040, 1.081),
        (0.028, 0.966, 0.034, 0.981, 0.038, 1.092),
        (0.029, 0.961, 0.035, 0.974, 0.039, 1.090),
        (0.020, 0.921, 0.024, 0.937, 0.021, 1.094)
    ]

    # 式(2)における係数
    if mode == 'い' or mode == '住戸全体を連続的に暖房する方式' or mode == '全館連続':
        a_R_Evp_i = table_3[region - 1][0]
        b_R_Evp_i = table_3[region - 1][1]
    elif mode == 'ろ':
        a_R_Evp_i = table_3[region - 1][2]
        b_R_Evp_i = table_3[region - 1][3]
    elif mode == 'は':
        a_R_Evp_i = table_3[region - 1][4]
        b_R_Evp_i = table_3[region - 1][5]
    else:
        raise ValueError(mode)

    return a_R_Evp_i * Q + b_R_Evp_i  # (2)

温水床暖房又は電気ヒーター床暖房を暖冷房区画$i$において採用しない場合は、1.0とする。 

If hot floor heating or electric heater floor heating is not adopted in heating / cooling section $ i $, it is 1.0.

$$\large
f_{R,Eqp,i} = a_{R,Eqp,i} \times R_{l,i} + 1 
$$

$R_{l,i}$：暖冷房区画$i$に設置された温水床暖房又は電気ヒーター床暖房の敷設率 / Rate of laying hot floor heating or electric heating floor heating installed in heating and cooling section $ i $  

In [ ]:
def get_f_R_Eqp_i(floorheating, mode, R_l_i):
    # 表 4 式(3)における係数
    table_4 = (-0.105, -0.137, -0.231)

    # 温水床暖房又は電気ヒーター床暖房を暖冷房区画݅において採用する場合は、式(3)により表されるものとし、
    # 温水床暖房又は電気ヒーター床暖房を暖冷房区画݅において採用しない場合は、1.0とする。
    if floorheating:

        # 式(3)における係数
        if mode == 'い':
            a_R_Eqp_i = table_4[0]
        elif mode == 'ろ':
            a_R_Eqp_i = table_4[1]
        elif mode == 'は':
            a_R_Eqp_i = table_4[2]
        else:
            raise ValueError(mode)

        return a_R_Eqp_i * R_l_i + 1.0  # (3)
    else:
        return 1.0

$$\large
r'_{Af} = \frac{A_{f}}{A'_{HCZ}}
$$

$A_{f}$：（当該住戸における）床暖房パネルの敷設面積(m<sup>2</sup>) / Flooring panel floor area (in the relevant unit)  
$A'_{HCZ}$：（当該住戸における）主たる居室における仮想床の床面積を除いた床面積(m<sup>2</sup>) / Floor area excluding the floor area of the virtual floor in the main room (in the relevant dwelling unit)  

<div style="text-align: center;font-weight: bold;">表　係数$a_{R,Eqp,i}$<br>Table: Coefficients</div>

<table>
<thead>
<tr>
<th colspan="3"><center>暖房方式及び運転方法の区分<br>Classification of heating method and operation method</center></th>
</tr>
<tr>
<th>（い）<br>(I)</th>
<th>（ろ）<br>(RO)</th>
<th>（は）<br>(HA)</th>
</tr>
<tr>
<th>「住宅全体を連続して暖房する方式」<br>"Continuous heating system of the whole house"</th>
<th>「居室のみを暖房する方式」<br>かつ「連続運転」の場合<br>In the case of "method to heat only the living room" and "continuous operation"</th>
<th>「居室のみを暖房する方式」<br>かつ「間歇運転」の場合<br>"Method to heat only the living room" and in the case of "intermittent operation"</th>
</tr>
</thead>

<tbody>
<tr>
<td>-0.105</td>
<td>-0.137</td>
<td>-0.231</td>
</tr>
</tbody>
</table>


暖冷房区画$i$に温水床暖房又は電気ヒーター床暖房を採用する場合は1.0とし、それ以外の場合は、下式により表される。 

In the case where hot floor heating or electric heater floor heating is adopted for the heating and cooling section $ i $, it is 1.0, and in the other cases, it is expressed by the following equation.

$$\large
f_{TD,i} = min(a_{TD,i} \times Q^2 + 1, f_{TD,max})
$$

<div style="text-align: center;font-weight: bold;">表　係数$a_{TD,i}$<br>Table: Coefficients $a_{TD,i}$</div>

<table>
<thead>
<tr>
<th rowspan="4">地域の区分<br>Segmentation of area</th>
    <th colspan="6"><center>暖房方式及び運転方法の区分<br>Classification of heating method and operation method</center></th>
</tr>
<tr>
<th colspan="2">（い）<br>(I)</th>
<th colspan="2">（ろ）<br>(RO)</th>
<th colspan="2">（は）<br>(HA)</th>
</tr>
<tr>
<th colspan="2">「住戸全体を連続的に<br>暖房する方式」<br>"Method to heat the entire dwelling unit continuously"</th>
<th colspan="2">「居室のみを暖房する方式」<br>かつ「連続運転」の場合<br>In the case of "method to heat only the living room" and "continuous operation"</th>
<th colspan="2">「居室のみを暖房する方式」<br>かつ「間歇運転」の場合<br>"Method to heat only the living room" and in the case of "intermittent operation"</tr>
<tr>
<th>$a_{TD,i}$</th>
<th>$f_{TD,max}$</th>
<th>$a_{TD,i}$</th>
<th>$f_{TD,max}$</th>
<th>$a_{TD,i}$</th>
<th>$f_{TD,max}$</th>
</tr>
</thead>

<tbody>
<tr>
<th>1</th>
<td>0.0157</td>
<td>1.0842</td>
<td>0.0163</td>
<td>1.0862</td>
<td>0.0176</td>
<td>1.0860</td>
</tr>
<tr>
<th>2</th>
<td>0.0157</td>
<td>1.0928</td>
<td>0.0163</td>
<td>1.0954</td>
<td>0.0176</td>
<td>1.0981</td>
</tr>
<tr>
<th>3</th>
<td>0.0097</td>
<td>1.1048</td>
<td>0.0101</td>
<td>1.1079</td>
<td>0.0110</td>
<td>1.1147</td>
</tr>
<tr>
<th>4</th>
<td>0.0063</td>
<td>1.1111</td>
<td>0.0066</td>
<td>1.1146</td>
<td>0.0072</td>
<td>1.1235</td>
</tr>
<tr>
<th>5</th>
<td>0.0045</td>
<td>1.1223</td>
<td>0.0047</td>
<td>1.1264</td>
<td>0.0053</td>
<td>1.1391</td>
</tr>
<tr>
<th>6</th>
<td>0.0045</td>
<td>1.1277</td>
<td>0.0047</td>
<td>1.1320</td>
<td>0.0053</td>
<td>1.1465</td>
</tr>
<tr>
<th>7</th>
<td>0.0014</td>
<td>1.1357</td>
<td>0.0015</td>
<td>1.1404</td>
<td>0.0017</td>
<td>1.1576</td>
</tr>
</tbody>
</table>


In [ ]:
# 暖冷房区画݅における上下温度分布を考慮した負荷補正係数
def get_f_TD_i(floorheating, region, mode, Q):
    # 表 5 式(5)における係数
    table_5 = [
        (0.0157, 1.0842, 0.0163, 1.0862, 0.0176, 1.0860),
        (0.0157, 1.0928, 0.0163, 1.0954, 0.0176, 1.0981),
        (0.0097, 1.1048, 0.0101, 1.1079, 0.0110, 1.1147),
        (0.0063, 1.1111, 0.0066, 1.1146, 0.0072, 1.1235),
        (0.0045, 1.1223, 0.0047, 1.1264, 0.0053, 1.1391),
        (0.0045, 1.1277, 0.0047, 1.1320, 0.0053, 1.1465),
        (0.0014, 1.1357, 0.0015, 1.1404, 0.0017, 1.1576)
    ]

    # 暖冷房区画iに温水床暖房又は電気ヒーター床暖房を採用する場合は1.0とし、それ以外の場合は、式(5)により表される。
    if floorheating:
        return 1.0
    else:

        if mode == 'い' or mode == '住戸全体を連続的に暖房する方式' or mode == '全館連続':
            a_TD_i = table_5[region - 1][0]
            f_TD_max = table_5[region - 1][1]
        elif mode == 'ろ':
            a_TD_i = table_5[region - 1][2]
            f_TD_max = table_5[region - 1][3]
        elif mode == 'は':
            a_TD_i = table_5[region - 1][4]
            f_TD_max = table_5[region - 1][5]
        else:
            raise ValueError(mode)

        return min(a_TD_i * Q ** 2 + 1, f_TD_max)  # (5)

$$\large
L_{CS,d,t,i} = L'_{CS,d,t,i}
$$

$$\large
L_{CL,d,t,i} = L'_{CL,d,t,i}
$$

$L_{CS,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の冷房顕熱負荷(MJ/h) / Cooling sensible heat load of heating / cooling section $ i $ at time $ t $ of date $ d $  
$L_{CL,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の冷房潜熱負荷(MJ/h) / Cooling latent heat load of heating / cooling section $ i $ at time $ t $ of date $ d $  
$L'_{CS,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の負荷補正前の冷房顕熱負荷(MJ/h) / Cooling sensible heat load before load correction of heating / cooling section $ i $ at time $ t $ of date $ d $  
$L'_{CL,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の負荷補正前の冷房潜熱負荷(MJ/h) / Cooling latent heat load before load correction of heating / cooling section $ i $ at time $ t $ of date $ d $  

In [ ]:
# 暖冷房区画iの 1 時間当たりの冷房顕熱負荷
def get_L_CS_d_t_i(region, A_A, A_MR, A_OR, mode, NV_MR, NV_OR, Q, mu_C, r_A_ufvnt, underfloor_insulation):
    from section11_1 import load_outdoor, get_Theta_ex
    outdoor = load_outdoor()
    Theta_ex_d_t = get_Theta_ex(region, outdoor)

    L_CS_d_t_i = get_L_dash_CS_d_t_i(region, A_A, A_MR, A_OR, mode, NV_MR, NV_OR, Q, mu_C, r_A_ufvnt,
                                     underfloor_insulation, Theta_ex_d_t)  # (6a)
    return L_CS_d_t_i


# 暖冷房区画iの 1 時間当たりの冷房潜熱負荷
def get_L_CL_d_t_i(region, A_A, A_MR, A_OR, mode, NV_MR, NV_OR, Q, mu_C):
    L_CL_d_t_i = get_L_dash_CL_d_t_i(region, A_A, A_MR, A_OR, mode, NV_MR, NV_OR, Q, mu_C)  # (6a)
    return L_CL_d_t_i

### 6．負荷補正前の暖冷房負荷 / Heating and cooling load before load correction

$$\large
L'_{H,d,t,i} = L'_{H,R,d,t,i} \times \frac{A_{HCZ,i}}{A_{HCZ,R,i}} - \Delta L'_{H,uf,d,t,i} - \Delta L'_{H,ass,d,t,i}
$$

$$\large
L'_{CS,d,t,i} = L'_{CS,R,d,t,i} \times \frac{A_{HCZ,i}}{A_{HCZ,R,i}} - \Delta L'_{CS,uf,d,t,i}
$$

$$\large
L'_{CL,d,t,i} = L'_{CL,R,d,t,i} \times \frac{A_{HCZ,i}}{A_{HCZ,R,i}}
$$

$L'_{H,R,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の標準住戸の負荷補正前の暖房負荷(MJ/h) / Heating load before load compensation of a standard dwelling unit $ i $ in a heating and cooling compartment $ i $ at a time $ t $ on a date $ d $  
$L'_{CS,R,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の標準住戸の負荷補正前の冷房顕熱負荷(MJ/h) / Cooling sensible heat load before load compensation of standard dwelling unit $ i $ in heating / cooling section $ i $ at time $ t $ on date $ d $  
$L'_{CL,R,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の標準住戸の負荷補正前の冷房潜熱負荷(MJ/h) / Cooling latent heat load before load compensation of standard dwelling unit of heating / cooling section $ i $ at time $ t $ of date $ d $  
$\Delta L'_{H,ass,d,t,i}$：日付$d$の時刻$t$における当該住戸の暖冷房区画$i$の空気集熱式太陽熱利用設備による暖房負荷削減量(MJ/h) / The amount of heating load reduction by the air collection solar heating equipment of heating / cooling section $ i $ of the relevant dwelling unit at time $ t $ of date $ d $  
$\Delta L'_{H,uf,d,t,i}$：日付$d$の時刻$t$における当該住戸の暖冷房区画$i$の床下空間を経由して外気を導入する換気方式による暖房負荷削減量(MJ/h) / The amount of heating load reduction by the ventilation system that introduces outside air via the floor space of the heating and cooling section $ i $ of the relevant dwelling unit at the time $ t $ of date $ d $  
$\Delta L'_{CS,uf,d,t,i}$：日付$d$の時刻$t$における当該住戸の暖冷房区画$i$の床下空間を経由して外気を導入する換気方式による冷房顕熱負荷削減量(MJ/h) / Cooling sensible heat load reduction amount by ventilation system which introduces outside air via underfloor space of heating and cooling section $ i $ of the dwelling unit at time $ t $ of date $ d $  
$A_{HCZ,i}$：暖冷房区画$i$の床面積(m<sup>2</sup>) / Floor area of heating / cooling section $ i $  
$A_{HCZ,R,i}$：標準住戸の暖冷房区画$i$の床面積(m<sup>2</sup>) / Floor area of heating / cooling section $ i $ of standard dwelling unit  

日付$d$の時刻$t$における当該住戸の暖冷房区画$i$の床下空間を経由して外気を導入する換気方式による暖房負荷削減量$\Delta L'_{H,uf,d,t,i}$ 及び日付$d$の時刻$t$における当該住戸の暖冷房区画$i$の床下空間を経由して外気を導入する換気方式による冷房顕熱負荷削減量$\Delta L'_{CS,uf,d,t,i}$ は、床下空間を経由して外気を導入する換気方式を採用しない場合はゼロとする。  
日付$d$の時刻$t$における当該住戸の暖冷房区画$i$の空気集熱式太陽熱利用設備による暖房負荷削減量$\Delta L'_{H,ass,d,t,i}$ は、空気集熱式太陽熱利用設備を採用しない場合はゼロとする。  

Heating load reduction amount $\Delta L'_{CS,uf,d,t,i}$ by ventilation system introducing outside air via heating and cooling division $ i $ under floor space of the dwelling unit at time $ t $ of date $ d $ and time $ t $ of date $ d $ The cooling sensible heat load reduction amount $\Delta L'_{CS,uf,d,t,i}$ by the ventilation system that introduces outside air via the floor space under the heating and cooling section $ i $ of the relevant dwelling unit in the house does not adopt the ventilation system that introduces the outside air via the floor space If it is zero.  
Heating load reduction amount by air-collecting type solar utilization equipment of heating / cooling section $ i $ of the relevant dwelling unit at time $ t $ of date $ d $ $ \Delta L '_ {H, ass, d, t, i} $ If no air collection solar utilization equipment is adopted, it shall be zero.

In [ ]:
# 負荷補正前の暖房負荷 (7a)
def get_L_dash_H_d_t_i(region, A_A, A_MR, A_OR, mode, TS, Q, mu_H, hex, etr_dash_t, r_A_ufvnt, underfloor_insulation,
                       Theta_ex_d_t, hotwater_use=None, supply_target=None, sol_region=None, P_alpha=None, P_beta=None,
                       A_col=None, V_fan_P0=None, m_fan_test=None, d0=None, d1=None, r_A_ufvnt_ass=None, ufv_insulation=None):
    # 標準住戸の負荷補正前の暖房負荷 (8)
    L_dash_H_R_d_t_i = calc_dash_H_R_d_t_i(A_A, A_MR, A_OR, Q, TS, etr_dash_t, hex, mode, mu_H, region)

    # 床面積按分及び床下換気・空気集熱式太陽熱利用設備による負荷削減
    L_dash_H_d_t_i = np.zeros((12, 24 * 365))
    for i in range(1, 13):
        # 床面積
        A_HCZ_R_i = get_A_HCZ_R_i(i)
        A_HCZ_i = get_A_HCZ_i(i, A_A, A_MR, A_OR)

        # 床下空間を経由して外気を導入する換気方式による暖房負荷削減
        delta_L_dash_H_uf_d_t_i = uf.get_delta_L_dash_H_uf_d_t_i(
            i=i,
            A_A=A_A,
            A_MR=A_MR,
            A_OR=A_OR,
            A_HCZ_i=A_HCZ_i,
            region=region,
            Q=Q,
            r_A_ufvnt=r_A_ufvnt,
            underfloor_insulation=underfloor_insulation,
            Theta_ex_d_t=Theta_ex_d_t,
            L_dash_H_R_d_t_i=L_dash_H_R_d_t_i
        )

        # 空気集熱式太陽熱利用設備を採用する場合
        if supply_target is not None:
            delta_L_dash_H_ass_d_t_i = ass.calc_delta_L_dash_H_ass_d_t_i(
                i=i,
                supply_target=supply_target,
                L_dash_H_R_d_t_i=L_dash_H_R_d_t_i,
                region=region,
                sol_region=sol_region,
                A_HCZ_i=A_HCZ_i,
                A_A=A_A,
                A_MR=A_MR,
                A_OR=A_OR,
                Q=Q,
                hotwater_use=hotwater_use,
                Theta_ex_d_t=Theta_ex_d_t,
                P_alpha=P_alpha,
                P_beta=P_beta,
                A_col=A_col,
                V_fan_P0=V_fan_P0,
                m_fan_test=m_fan_test,
                d0=d0,
                d1=d1,
                ufv_insulation=ufv_insulation,
                r_A_ufvnt=r_A_ufvnt_ass
            )
        else:
            delta_L_dash_H_ass_d_t_i = np.zeros(24 * 365)

        # (7a)
        L_dash_H_d_t_i[i - 1, :] = L_dash_H_R_d_t_i[i - 1, :] * (A_HCZ_i / A_HCZ_R_i)

        f = L_dash_H_d_t_i[i - 1, :] > 0
        L_dash_H_d_t_i[i - 1, f] = L_dash_H_d_t_i[i - 1, f] - delta_L_dash_H_uf_d_t_i[f]

        f = L_dash_H_d_t_i[i - 1, :] > 0
        L_dash_H_d_t_i[i - 1, f] = L_dash_H_d_t_i[i - 1, f] - delta_L_dash_H_ass_d_t_i[f]

    return L_dash_H_d_t_i, L_dash_H_R_d_t_i



# 負荷補正前の冷房顕熱負荷 (7b)
def get_L_dash_CS_d_t_i(region, A_A, A_MR, A_OR, mode, NV_MR, NV_OR, Q, mu_C, r_A_ufvnt, underfloor_insulation,
                        Theta_ex_d_t):
    L_dash_CS_R_d_t_i = np.zeros((12, 24 * 365))
    L_dash_CS_d_t_i = np.zeros((12, 24 * 365))

    for i in range(1, 13):
        if mode[i - 1] is not None:
            L_dash_CS_R_d_t_i[i - 1, :] = get_L_dash_CS_R_d_t_i(region, mode[i - 1], NV_MR, NV_OR, Q, mu_C, i)

    # 床面積による按分と床下空間を経由して外気を導入する換気方式による冷房顕熱負荷削減
    for i in range(1, 13):
        if mode[i - 1] is not None:
            # 床面積
            A_HCZ_R_i = get_A_HCZ_R_i(i)
            A_HCZ_i = get_A_HCZ_i(i, A_A, A_MR, A_OR)

            # 床下空間を経由して外気を導入する換気方式による冷房顕熱負荷削減
            delta_L_dash_CS_uf_d_t_i = uf.get_delta_L_dash_CS_R_d_t_i(i, region, Q, r_A_ufvnt, underfloor_insulation,
                                                                      A_A, A_MR, A_OR, A_HCZ_i, Theta_ex_d_t,
                                                                      L_dash_CS_R_d_t_i)

            L_dash_CS_d_t_i[i - 1, :] = L_dash_CS_R_d_t_i[i - 1, :] * (
                    A_HCZ_i / A_HCZ_R_i) - delta_L_dash_CS_uf_d_t_i  # (7b)

    return L_dash_CS_d_t_i


# 負荷補正前の冷房潜熱負荷 (7c)
def get_L_dash_CL_d_t_i(region, A_A, A_MR, A_OR, mode, NV_MR, NV_OR, Q, mu_C):
    L_dash_CL_R_d_t_i = np.zeros((12, 24 * 365))
    L_dash_CL_d_t_i = np.zeros((12, 24 * 365))

    for i in range(1, 13):
        if mode[i - 1] is not None:
            # 床面積
            A_HCZ_R_i = get_A_HCZ_R_i(i)
            A_HCZ_i = get_A_HCZ_i(i, A_A, A_MR, A_OR)

            L_dash_CL_R_d_t_i[i - 1, :] = get_L_dash_CL_R_d_t_i(region, mode[i - 1], NV_MR, NV_OR, Q, mu_C, i)

            L_dash_CL_d_t_i[i - 1, :] = L_dash_CL_R_d_t_i[i - 1, :] * (A_HCZ_i / A_HCZ_R_i)  # (7c)

    return L_dash_CL_d_t_i

### 7．標準住戸の負荷補正前の暖冷房負荷 / Heating and cooling load before load compensation of standard dwelling unit

#### 7.1　標準住戸の負荷補正前の暖房負荷 / Heating load before load compensation of standard dwelling unit

蓄熱の利用なしの場合： 

With no use of heat storage:

$$\large
L'_{H,R,d,t,i} = L'_{H,R,TS_0,d,t,i}
$$

蓄熱の利用ありの場合： 

With the use of heat storage:

$$\large
L'_{H,R,d,t,i} = L'_{H,R,TS_1,d,t,i}
$$

$L'_{H,R,TS_l,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の蓄熱の利用の程度$TS_l$の標準住戸の負荷補正前の暖房負荷(MJ/h) / Degree of heat storage utilization of heating / cooling compartment $ i $ at time $ t $ of date $ d $ Heating load before load compensation of standard dwelling unit $ TS_l $  

<div style="text-align: center;font-weight: bold;">表　蓄熱の採用の可否<br>Table: Availability of heat storage</div>

<table>
<thead>
<tr>
<th rowspan="2">地域区分<br>Regional division</th>
<th colspan="5">暖房期の日射地域区分<br>Solar radiation area division of heating season</th>
</tr>
<tr>
<th>H1</th>
<th>H2</th>
<th>H3</th>
<th>H4</th>
<th>H5</th>
</tr>
</thead>

<tbody>
<tr>
<th>1</th>
<td>不可<br>Impossible</td>
<td>不可<br>Impossible</td>
<td>可<br>Possible</td>
<td>可<br>Possible</td>
<td>可<br>Possible</td>
</tr>
<tr>
<th>2</th>
<td>不可<br>Impossible</td>
<td>不可<br>Impossible</td>
<td>可<br>Possible</td>
<td>可<br>Possible</td>
<td>可<br>Possible</td>
</tr>
<tr>
<th>3</th>
<td>不可<br>Impossible</td>
<td>不可<br>Impossible</td>
<td>可<br>Possible</td>
<td>可<br>Possible</td>
<td>可<br>Possible</td>
</tr>
<tr>
<th>4</th>
<td>不可<br>Impossible</td>
<td>不可<br>Impossible</td>
<td>可<br>Possible</td>
<td>可<br>Possible</td>
<td>可<br>Possible</td>
</tr>
<tr>
<th>5</th>
<td>不可<br>Impossible</td>
<td>不可<br>Impossible</td>
<td>可<br>Possible</td>
<td>可<br>Possible</td>
<td>可<br>Possible</td>
</tr>
<tr>
<th>6</th>
<td>不可<br>Impossible</td>
<td>不可<br>Impossible</td>
<td>不可<br>Impossible</td>
<td>可<br>Possible</td>
<td>可<br>Possible</td>
</tr>
<tr>
<th>7</th>
<td>不可<br>Impossible</td>
<td>不可<br>Impossible</td>
<td>不可<br>Impossible</td>
<td>可<br>Possible</td>
<td>可<br>Possible</td>
</tr>
</tbody>
</table>


In [ ]:
# 標準住戸の負荷補正前の暖房負荷 (8)
def calc_dash_H_R_d_t_i(A_A, A_MR, A_OR, Q, TS, etr_dash_t, hex, mode, mu_H, region):
    L_dash_H_R_d_t_i = np.zeros((12, 24 * 365))

    # 標準住戸の負荷補正前の暖房負荷を全居室求める
    for i in range(1, 13):
        # 床面積
        A_HCZ_i = get_A_HCZ_i(i, A_A, A_MR, A_OR)

        if mode[i - 1] is not None and A_HCZ_i > 0:
            L_dash_H_R_d_t_i[i - 1, :] = get_L_dash_H_R_d_t_i(
                region=region,
                mode=mode[i - 1],
                TS=TS,
                Q=Q,
                mu_H=mu_H,
                hex=hex,
                etr_dash_t=etr_dash_t,
                i=i
            )
        else:
            L_dash_H_R_d_t_i[i - 1, :] = np.zeros(24 * 365)
    return L_dash_H_R_d_t_i


# 標準住戸の負荷補正前の暖房負荷 (8)
def get_L_dash_H_R_d_t_i(region, mode, TS, Q, mu_H, hex, etr_dash_t, i):
    if not TS:
        # 蓄熱の利用なしの場合
        L_dash_H_R_TSl_d_t_i = get_L_dash_H_R_TSl_d_t_i(region, mode, 1, Q, mu_H, hex, etr_dash_t, i)  # (8-1)
    else:
        # 蓄熱の利用ありの場合
        L_dash_H_R_TSl_d_t_i = get_L_dash_H_R_TSl_d_t_i(region, mode, 2, Q, mu_H, hex, etr_dash_t, i)  # (8-2)

    return L_dash_H_R_TSl_d_t_i


# 表 6 蓄熱の採用の可否
table_6 = [
    ('不可', '不可', '可', '可', '可'),
    ('不可', '不可', '可', '可', '可'),
    ('不可', '不可', '可', '可', '可'),
    ('不可', '不可', '可', '可', '可'),
    ('不可', '不可', '可', '可', '可'),
    ('不可', '不可', '不可', '可', '可'),
    ('不可', '不可', '不可', '可', '可')
]

1～7 地域の場合

In the case of 1-7 areas

$$\large
L'_{H,R,TS_l,d,t,i} = \left\{ \begin{array}{ll}
    \frac{Q_{HEXC}-Q_2}{Q_1 - Q_2} \times L'_{H,R,TS_l,Q_1,d,t,i} + \frac{Q_{HEXC}-Q_1}{Q_2 - Q_1} \times L'_{H,R,TS_l,Q_2,d,t,i} & (Q \ge Q_2の場合) \\
    \frac{Q_{HEXC}-Q_3}{Q_2 - Q_3} \times L'_{H,R,TS_l,Q_2,d,t,i} + \frac{Q_{HEXC}-Q_2}{Q_3 - Q_2} \times L'_{H,R,TS_l,Q_3,d,t,i} & (Q_2 > Q \ge Q_3の場合) \\
    \frac{Q_{HEXC}-Q_4}{Q_3 - Q_4} \times L'_{H,R,TS_l,Q_3,d,t,i} + \frac{Q_{HEXC}-Q_3}{Q_4 - Q_3} \times L'_{H,R,TS_l,Q_4,d,t,i} & (Q_3 > Qの場合) \\
\end{array}
\right.
$$

8 地域の場合

In the case of 8 areas

$$\large
L'_{H,R,TS_l,d,t,i} = \left\{ \begin{array}{ll}
    \frac{Q_{HEXC}-Q_2}{Q_1 - Q_2} \times L'_{H,R,TS_l,Q_1,d,t,i} + \frac{Q_{HEXC}-Q_1}{Q_2 - Q_1} \times L'_{H,R,TS_l,Q_2,d,t,i} & (Q \ge Q_2の場合) \\
    \frac{Q_{HEXC}-Q_3}{Q_2 - Q_3} \times L'_{H,R,TS_l,Q_2,d,t,i} + \frac{Q_{HEXC}-Q_2}{Q_3 - Q_2} \times L'_{H,R,TS_l,Q_3,d,t,i} & (Q_2 > Qの場合) \\
\end{array}
\right.
$$

$Q_{HEXC}$：熱交換型換気設備による暖房負荷低減を考慮した補正熱損失係数(W/m<sup>2</sup>K) / Corrected heat loss coefficient considering heating load reduction by heat exchange type ventilation equipment  
$L'_{H,R,TS_l,Q_j,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の蓄熱の利用の程度$TS_l$の熱損失係数$Q_{j}$の標準住戸の負荷補正前の暖房負荷(MJ/h) / Degree of heat storage utilization of heating / cooling division $ i $ at time $ t $ of date $ d $ Heat loss coefficient of standard dwelling unit of $ Q_ {j} $, heat loss coefficient of $ TS_l $  
$Q_{j}$：断熱性能の区分$j$の熱損失係数(W/(m<sup>2</sup>K)) / Heat loss coefficient of division $ j $ of thermal insulation performance  

In [ ]:
# 暖房負荷のQ値按分 (9)
def get_L_dash_H_R_TSl_d_t_i(region, mode, l, Q, mu_H, hex, etr_dash_t, i):
    # 熱交換型換気設備による暖房負荷低減を考慮した補正熱損失係数
    Q_HEXC = get_Q_HEXC(region, Q, hex, etr_dash_t)

    if region in [1, 2, 3, 4, 5, 6, 7]:

        Q1 = get_Q_j(region, 1)
        Q2 = get_Q_j(region, 2)
        Q3 = get_Q_j(region, 3)
        Q4 = get_Q_j(region, 4)

        # (9a)
        if Q_HEXC == Q1:
            return get_L_dash_H_R_TSl_Qj_d_t_i(region, mode, l, 1, mu_H, hex, i)
        elif Q_HEXC == Q2:
            return get_L_dash_H_R_TSl_Qj_d_t_i(region, mode, l, 2, mu_H, hex, i)
        elif Q_HEXC == Q3:
            return get_L_dash_H_R_TSl_Qj_d_t_i(region, mode, l, 3, mu_H, hex, i)
        elif Q_HEXC == Q4:
            return get_L_dash_H_R_TSl_Qj_d_t_i(region, mode, l, 4, mu_H, hex, i)
        elif Q_HEXC >= Q2:
            return (Q_HEXC - Q2) / (Q1 - Q2) * get_L_dash_H_R_TSl_Qj_d_t_i(region, mode, l, 1, mu_H, hex, i) \
                   + (Q_HEXC - Q1) / (Q2 - Q1) * get_L_dash_H_R_TSl_Qj_d_t_i(region, mode, l, 2, mu_H, hex, i)
        elif Q2 > Q_HEXC and Q_HEXC >= Q3:
            return (Q_HEXC - Q3) / (Q2 - Q3) * get_L_dash_H_R_TSl_Qj_d_t_i(region, mode, l, 2, mu_H, hex, i) \
                   + (Q_HEXC - Q2) / (Q3 - Q2) * get_L_dash_H_R_TSl_Qj_d_t_i(region, mode, l, 3, mu_H, hex, i)
        elif Q3 > Q_HEXC:
            return (Q_HEXC - Q4) / (Q3 - Q4) * get_L_dash_H_R_TSl_Qj_d_t_i(region, mode, l, 3, mu_H, hex, i) \
                   + (Q_HEXC - Q3) / (Q4 - Q3) * get_L_dash_H_R_TSl_Qj_d_t_i(region, mode, l, 4, mu_H, hex, i)
        else:
            raise ValueError(Q_HEXC)

    elif region in [8]:

        Q1 = get_Q_j(region, 1)
        Q2 = get_Q_j(region, 2)
        Q3 = get_Q_j(region, 3)

        # (9b)
        if Q == Q1:
            return get_L_dash_H_R_TSl_Qj_d_t_i(region, mode, l, 1, mu_H, hex, i)
        elif Q == Q2:
            return get_L_dash_H_R_TSl_Qj_d_t_i(region, mode, l, 2, mu_H, hex, i)
        elif Q == Q3:
            return get_L_dash_H_R_TSl_Qj_d_t_i(region, mode, l, 3, mu_H, hex, i)
        elif Q >= Q2:
            return (Q_HEXC - Q2) / (Q1 - Q2) * get_L_dash_H_R_TSl_Qj_d_t_i(region, mode, l, 1, mu_H, hex, i) \
                   + (Q_HEXC - Q1) / (Q2 - Q1) * get_L_dash_H_R_TSl_Qj_d_t_i(region, mode, l, 2, mu_H, hex, i)
        elif Q2 > Q and Q >= Q3:
            return (Q_HEXC - Q3) / (Q2 - Q3) * get_L_dash_H_R_TSl_Qj_d_t_i(region, mode, l, 2, mu_H, hex, i) \
                   + (Q_HEXC - Q2) / (Q3 - Q2) * get_L_dash_H_R_TSl_Qj_d_t_i(region, mode, l, 3, mu_H, hex, i)
        else:
            raise ValueError(Q)

    else:
        raise ValueError(region)

$$\large
L'_{H,R,TS_l,Q_j,d,t,i} = \left\{ \begin{array}{ll}
    \frac{\mu_H - \mu_{H,j,2}}{\mu_{H,j,1} - \mu_{H,j,2}} \times L'_{H,R,TS_l,Q_j,\mu_{H,j,1},d,t,i} + \frac{\mu_{H} - \mu_{H,j,1}}{\mu_{H,j,2} - \mu_{H,j,1}} \times L'_{H,R,TS_l,Q_j,\mu_{H,j,2},d,t,i} & (\mu_{H} < \mu_{H,j,2}) \\
    \frac{\mu_H - \mu_{H,j,3}}{\mu_{H,j,2} - \mu_{H,j,3}} \times L'_{H,R,TS_l,Q_j,\mu_{H,j,2},d,t,i} + \frac{\mu_{H} - \mu_{H,j,2}}{\mu_{H,j,3} - \mu_{H,j,2}} \times L'_{H,R,TS_l,Q_j,\mu_{H,j,3},d,t,i} & (\mu_{H,j,2} \le \mu_{H}) \\
\end{array}
\right.
$$

$L'_{H,R,TS_l,Q_j,\mu_{H,j,k},d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の蓄熱の利用の程度$TS_l$の熱損失係数$Q_{j}$かつ暖房期の日射取得係数$\mu_{H,j,k}$の標準住戸の負荷補正前の暖房負荷(MJ/h) / The standard heat load before load correction of the standard dwelling unit with the heat loss coefficient $Q_{j}$ of the degree $TS_l$ of utilization of heat storage utilization of the heating and cooling compartment $ i $ at the time $ t $ of date $ d $ and the solar radiation acquisition coefficient $\mu_{H,j,k}$ of the heating period  
$\mu_H$：暖房期の日射取得係数 / Solar radiation acquisition factor of heating season  
$\mu_{H,j,k}$：断熱性能の区分$j$における日射取得性能の区分$k$の暖房期の日射取得係数 / Division of solar radiation acquisition performance in division $ j $ of thermal insulation performance Division of solar radiation acquisition coefficient of heating season of $ k $  

In [ ]:
# 暖房負荷 μ値按分 (10)
def get_L_dash_H_R_TSl_Qj_d_t_i(region, mode, l, j, mu_H, hex, i):
    mu_H_j_1 = get_mu_H_j_k(region, j, 1)
    mu_H_j_2 = get_mu_H_j_k(region, j, 2)
    mu_H_j_3 = get_mu_H_j_k(region, j, 3)

    if mu_H < mu_H_j_2:
        return (mu_H - mu_H_j_2) / (mu_H_j_1 - mu_H_j_2) * get_L_dash_H_R_TSl_Qj_muH_j_k_d_t_i(region, mode, l, j, 1,
                                                                                               hex, i) \
               + (mu_H - mu_H_j_1) / (mu_H_j_2 - mu_H_j_1) * get_L_dash_H_R_TSl_Qj_muH_j_k_d_t_i(region, mode, l, j, 2,
                                                                                                 hex, i)
    elif mu_H_j_2 <= mu_H:
        return (mu_H - mu_H_j_3) / (mu_H_j_2 - mu_H_j_3) * get_L_dash_H_R_TSl_Qj_muH_j_k_d_t_i(region, mode, l, j, 2,
                                                                                               hex, i) \
               + (mu_H - mu_H_j_2) / (mu_H_j_3 - mu_H_j_2) * get_L_dash_H_R_TSl_Qj_muH_j_k_d_t_i(region, mode, l, j, 3,
                                                                                                 hex, i)
    else:
        raise ValueError(mu_H)

<div style="text-align: center;font-weight: bold;">表　断熱性能の区分$j$の熱損失係数$Q_{j}$<br>Table: Heat loss coefficient of insulation performance division $ j $</div>

<table>
<thead>
<tr>
<th rowspan="2" colspan="2"></th>
    <th colspan="8"><center>地域の区分<br>Segmentation of area</center></th>
</tr>
<tr>
<th>1</th>
<th>2</th>
<th>3</th>
<th>4</th>
<th>5</th>
<th>6</th>
<th>7</th>
<th>8</th>
</tr>
</thead>

<tbody>
<tr>
<th rowspan="4">断熱性能の区分$j$<br>Classification $j$<br>of thermal insulation performance</th>
<th>1</th>
<td>2.8</td>
<td>2.8</td>
<td>4.0</td>
<td>4.7</td>
<td>5.19</td>
<td>5.19</td>
<td>8.27</td>
<td>8.27</td>
</tr>
<tr>
<th>2</th>
<td>1.8</td>
<td>1.8</td>
<td>2.7</td>
<td>3.3</td>
<td>4.2</td>
<td>4.2</td>
<td>4.59</td>
<td>8.01</td>
</tr>
<tr>
<th>3</th>
<td>1.6</td>
<td>1.6</td>
<td>1.9</td>
<td>2.4</td>
<td>2.7</td>
<td>2.7</td>
<td>2.7</td>
<td>3.7</td>
</tr>
<tr>
<th>4</th>
<td>1.4</td>
<td>1.4</td>
<td>1.4</td>
<td>1.9</td>
<td>1.9</td>
<td>1.9</td>
<td>1.9</td>
<td>3.7</td>
</tr>
</tbody>
</table>

In [ ]:
# 断熱性能の区分j݆の熱損失係数ܳQ_j
def get_Q_j(region, j):
    # 表 7 断熱性能の区分݆jの熱損失係数Qj
    table_7 = [
        (2.8, 2.8, 4.0, 4.7, 5.19, 5.19, 8.27, 8.27),
        (1.8, 1.8, 2.7, 3.3, 4.2, 4.2, 4.59, 8.01),
        (1.6, 1.6, 1.9, 2.4, 2.7, 2.7, 2.7, 3.7),
        (1.4, 1.4, 1.4, 1.9, 1.9, 1.9, 1.9, 3.7)
    ]

    # 断熱性能の区分݆の熱損失係数Q_j(j=1-4)は地域の区分に応じて表 7 により表される。
    return table_7[j - 1][region - 1]

<div style="text-align: center;font-weight: bold;">表　断熱性能の区分$j$における日射取得性能の区分$k$の<br>暖房期の日射取得係数$\mu_{H,j,k}$及び冷房期の日射取得係数$\mu_{C,j,k}$<br>Table: Division $ k $ of solar radiation acquisition performance in division $ j $ of insulation performance <br> Solar acquisition coefficient $ \mu_ {H, j, k} $ of heating period and solar radiation acquisition coefficient $ \mu_ {C,  j, k} $of cooling period</div>
<div style="text-align: center;">（上段：暖房期の日射取得係数$\mu_{H,j,k}$　下段：冷房期の日射取得係数$\mu_{C,j,k}$）<br>(Upper: Solar radiation acquisition coefficient $ \mu_ {H, j, k} $ for heating season, Lower: Solar radiation acquisition coefficient $ \mu_ {C, j, k} $ for cooling season)</div>

<table>
<thead>
<tr>
<th rowspan="2">断熱性能の<br>区分$j$<br>Classification $j$ <br>of thermal insulation performance</th>
<th rowspan="2">日射取得性能の<br>区分$k$<br>Classification $k$ <br>of solar radiation acquisition performance</th>
    <th colspan="8"><center>地域の区分<br>Segmentation of area</center></th>
</tr>
<tr>
<th>1</th><th>2</th><th>3</th><th>4</th><th>5</th><th>6</th><th>7</th><th>8</th>
</tr>
</thead>

<tbody>
<tr>
<th>1</th><th>1</th>
<td>0.029<br>0.021</td>
<td>0.027<br>0.022</td>
<td>0.044<br>0.036</td>
<td>0.048<br>0.039</td>
<td>0.062<br>0.050</td>
<td>0.061<br>0.048</td>
<td>0.129<br>0.106</td>
<td>-<br>0.110</td>
</tr>
<tr>
<th>1</th><th>2</th>
<td>0.079<br>0.052</td>
<td>0.074<br>0.052</td>
<td>0.091<br>0.065</td>
<td>0.112<br>0.080</td>
<td>0.138<br>0.095</td>
<td>0.134<br>0.090</td>
<td>0.206<br>0.146</td>
<td>-<br>0.154</td>
</tr>
<tr>
<th>1</th><th>3</th>
<td>0.115<br>0.071</td>
<td>0.106<br>0.071</td>
<td>0.123<br>0.083</td>
<td>0.161<br>0.107</td>
<td>0.197<br>0.124</td>
<td>0.191<br>0.117</td>
<td>0.268<br>0.172</td>
<td>-<br>0.184</td>
</tr>
<tr>
<th>2</th><th>1</th>
<td>0.029<br>0.021</td>
<td>0.027<br>0.022</td>
<td>0.040<br>0.032</td>
<td>0.046<br>0.037</td>
<td>0.057<br>0.044</td>
<td>0.056<br>0.043</td>
<td>0.063<br>0.046</td>
<td>-<br>0.129</td>
</tr>
<tr>
<th>2</th><th>2</th>
<td>0.075<br>0.049</td>
<td>0.070<br>0.049</td>
<td>0.087<br>0.061</td>
<td>0.102<br>0.072</td>
<td>0.132<br>0.089</td>
<td>0.128<br>0.085</td>
<td>0.140<br>0.086</td>
<td>-<br>0.174</td>
</tr>
<tr>
<th>2</th><th>3</th>
<td>0.109<br>0.068</td>
<td>0.101<br>0.068</td>
<td>0.119<br>0.079</td>
<td>0.142<br>0.094</td>
<td>0.191<br>0.119</td>
<td>0.185<br>0.112</td>
<td>0.202<br>0.111</td>
<td>-<br>0.204</td>
</tr>
<tr>
<th>3</th><th>1</th>
<td>0.025<br>0.019</td>
<td>0.024<br>0.019</td>
<td>0.030<br>0.023</td>
<td>0.033<br>0.026</td>
<td>0.038<br>0.027</td>
<td>0.037<br>0.026</td>
<td>0.038<br>0.025</td>
<td>-<br>0.023</td>
</tr>
<tr>
<th>3</th><th>2</th>
<td>0.071<br>0.046</td>
<td>0.066<br>0.046</td>
<td>0.072<br>0.049</td>
<td>0.090<br>0.061</td>
<td>0.104<br>0.066</td>
<td>0.101<br>0.062</td>
<td>0.107<br>0.059</td>
<td>-<br>0.068</td>
</tr>
<tr>
<th>3</th><th>3</th>
<td>0.106<br>0.065</td>
<td>0.098<br>0.065</td>
<td>0.104<br>0.067</td>
<td>0.130<br>0.082</td>
<td>0.153<br>0.090</td>
<td>0.148<br>0.084</td>
<td>0.158<br>0.080</td>
<td>-<br>0.098</td>
</tr>
<tr>
<th>4</th><th>1</th>
<td>0.024<br>0.017</td>
<td>0.022<br>0.017</td>
<td>0.022<br>0.017</td>
<td>0.026<br>0.019</td>
<td>0.030<br>0.021</td>
<td>0.029<br>0.020</td>
<td>0.030<br>0.019</td>
<td>-<br>0.019</td>
</tr>
<tr>
<th>4</th><th>2</th>
<td>0.070<br>0.045</td>
<td>0.065<br>0.045</td>
<td>0.065<br>0.043</td>
<td>0.078<br>0.052</td>
<td>0.090<br>0.056</td>
<td>0.087<br>0.053</td>
<td>0.092<br>0.050</td>
<td>-<br>0.050</td>
</tr>
<tr>
<th>4</th><th>3</th>
<td>0.104<br>0.063</td>
<td>0.096<br>0.063</td>
<td>0.096<br>0.060</td>
<td>0.116<br>0.072</td>
<td>0.137<br>0.078</td>
<td>0.132<br>0.073</td>
<td>0.141<br>0.070</td>
<td>-<br>0.065</td>
</tr>
</tbody>
</table>


In [ ]:
# 表 8 断熱性能の区分݆における日射取得性能の区分݇の暖房期の日射取得係数mu_H_j_k及び冷房期の日射取得係数mu_C_j_k
table_8 = [
    (0.029, 0.027, 0.044, 0.048, 0.062, 0.061, 0.129, 0),
    (0.021, 0.022, 0.036, 0.039, 0.050, 0.048, 0.106, 0.11),
    (0.079, 0.074, 0.091, 0.112, 0.138, 0.134, 0.206, 0),
    (0.052, 0.052, 0.065, 0.080, 0.095, 0.090, 0.146, 0.154),
    (0.115, 0.106, 0.123, 0.161, 0.197, 0.191, 0.268, 0),
    (0.071, 0.071, 0.083, 0.107, 0.124, 0.117, 0.172, 0.184),
    (0.029, 0.027, 0.040, 0.046, 0.057, 0.056, 0.063, 0),
    (0.021, 0.022, 0.032, 0.037, 0.044, 0.043, 0.046, 0.129),
    (0.075, 0.070, 0.087, 0.102, 0.132, 0.128, 0.140, 0),
    (0.049, 0.049, 0.061, 0.072, 0.089, 0.085, 0.086, 0.174),
    (0.109, 0.101, 0.119, 0.142, 0.191, 0.185, 0.202, 0),
    (0.068, 0.068, 0.079, 0.094, 0.119, 0.112, 0.111, 0.204),
    (0.025, 0.024, 0.030, 0.033, 0.038, 0.037, 0.038, 0),
    (0.019, 0.019, 0.023, 0.026, 0.027, 0.026, 0.025, 0.023),
    (0.071, 0.066, 0.072, 0.090, 0.104, 0.101, 0.107, 0),
    (0.046, 0.046, 0.049, 0.061, 0.066, 0.062, 0.059, 0.068),
    (0.106, 0.098, 0.104, 0.130, 0.153, 0.148, 0.158, 0),
    (0.065, 0.065, 0.067, 0.082, 0.090, 0.084, 0.080, 0.098),
    (0.024, 0.022, 0.022, 0.026, 0.030, 0.029, 0.030, 0),
    (0.017, 0.017, 0.017, 0.019, 0.021, 0.020, 0.019, 0.019),
    (0.070, 0.065, 0.065, 0.078, 0.090, 0.087, 0.092, 0),
    (0.045, 0.045, 0.043, 0.052, 0.056, 0.053, 0.050, 0.050),
    (0.104, 0.096, 0.096, 0.116, 0.137, 0.132, 0.141, 0),
    (0.063, 0.063, 0.060, 0.072, 0.078, 0.073, 0.070, 0.065)
]

# 断熱性能の区分݆jにおける日射取得性能の区分k݇の暖房期の日射取得係数 mu_H_j_k
def get_mu_H_j_k(region, j, k):
    return table_8[(j - 1) * 6 + (k - 1) * 2 + 0][region - 1]


# 断熱性能の区分݆jにおける日射取得性能の区分k݇の冷房期の日射取得係数 mu_C_j_k
def get_mu_C_j_k(region, j, k):
    return table_8[(j - 1) * 6 + (k - 1) * 2 + 1][region - 1]

#### 7.2　標準住戸の負荷補正前の冷房負荷 / Cooling load before load compensation of standard dwelling unit

$$\large
L'_{CS,R,d,t,i} = \left\{ \begin{array}{ll}
    (\frac{NV-NV_2}{NV_1-NV_2} \times L'_{CS,R,NV_1,d,t,i} + \frac{NV-NV_1}{NV_2-NV_1} \times L'_{CS,R,NV_2,d,t,i}) & (NV < NV_2の場合) \\
    (\frac{NV-NV_3}{NV_2-NV_3} \times L'_{CS,R,NV_2,d,t,i} + \frac{NV-NV_2}{NV_3-NV_2} \times L'_{CS,R,NV_3,d,t,i}) & (NV_2 \le NVの場合) \\
\end{array}
\right.
$$

$$\large
L'_{CL,R,d,t,i} = \left\{ \begin{array}{ll}
    \frac{NV-NV_2}{NV_1-NV_2} \times L'_{CL,R,NV_1,d,t,i} + \frac{NV-NV_1}{NV_2-NV_1} \times L'_{CL,R,NV_2,d,t,i} & (NV < NV_2の場合) \\
    \frac{NV-NV_3}{NV_2-NV_3} \times L'_{CL,R,NV_2,d,t,i} + \frac{NV-NV_2}{NV_3-NV_2} \times L'_{CL,R,NV_3,d,t,i} & (NV_2 \le NVの場合) \\
\end{array}
\right.
$$

$NV$：通風の利用における相当換気回数(1/h) / Equivalent ventilation frequency in the use of ventilation  
$NV_l$：通風の利用に関する区分$l$の通風の利用における相当換気回数(1/h) / The equivalent ventilation frequency in the use of ventilation of division $ l $ concerning the use of ventilation  
$L'_{CS,R,NV_l,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の通風の利用における相当換気回数$NV_l$の標準住戸の負荷補正前の冷房顕熱負荷(MJ/h) / Cooling sensible heat load before load compensation of standard dwellings of equivalent ventilation frequency $ NV_l $ in utilization of ventilation of heating / cooling section $ i $ at time $ t $ of date $ d $  
$L'_{CL,R,NV_l,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の通風の利用における相当換気回数$NV_l$の標準住戸の負荷補正前の冷房潜熱負荷(MJ/h) / Cooling latent heat load before load compensation of standard dwellings with equivalent ventilation frequency $ NV_l $ in utilization of ventilation of heating / cooling section $ i $ at time $ t $ of date $ d $  

通風の利用における相当換気回数$NV$は、「住戸全体を連続的に冷房する方式」の場合はすべての暖冷房区画（$i = 1～12$）において0.0回/h とする。 

The equivalent ventilation frequency $ NV $ in the use of ventilation is 0.0 times / h in all heating and cooling sections ($ i = 1 to 12 $) in the case of the “system for continuously cooling the entire dwelling unit”.

主たる居室において通風の利用が「通風を利用しない」に該当する場合は暖冷房区画$i = 1$における通風の利用における相当換気回数$NV$は0.0回/h とし、通風の利用が「通風を利用する（換気回数5回/h 相当以上）」に該当する場合は暖冷房区画$i = 1$における通風の利用における相当換気回数$NV$は5.0回/h とし、通風の利用が「通風を利用する（換気回数20回/h 相当以上）」に該当する場合は暖冷房区画$i = 1$における通風の利用における相当換気回数$NV$は20.0回/h とする。  

If the use of ventilation is "do not use ventilation" in the main room, the equivalent ventilation frequency $ NV $ in the utilization of ventilation in the heating and cooling section $ i = 1 $ is 0.0 times / h, and the utilization of ventilation is "ventilation" In the case where it corresponds to the use of (ventilation frequency 5 times / h equivalent or more), the equivalent ventilation frequency $ NV $ in the utilization of ventilation in heating / cooling section $ i = 1 $ is 5.0 times / h, and utilization of ventilation is " In the case of using ventilation (20 times of ventilation / h or more equivalent), the equivalent ventilation frequency $ NV $ for using ventilation in heating / cooling section $ i = 1 $ is 20.0 times / h.

その他の居室において通風の利用が「通風を利用しない」に該当する場合は暖冷房区画$i = 2～5$における通風の利用における相当換気回数$NV$は0.0回/h とし、通風の利用が「通風を利用する（換気回数5回/h 相当以上）」に該当する場合は暖冷房区画$i = 2～5$における通風の利用における相当換気回数$NV$は5.0回/h とし、通風の利用が「通風を利用する（換気回数20回/h 相当以上）」に該当する場合は暖冷房区画$i = 2～5$における通風の利用における相当換気回数$NV$は20.0回/h とする。 

If the use of ventilation is "do not use ventilation" in the other rooms, the equivalent ventilation frequency $ NV $ in the utilization of ventilation in the heating and cooling section $ i = 2 to 5 $ is set to 0.0 times / h, and the use of ventilation In the case of using “ventilation” (ventilation frequency 5 times / h or more equivalent), the equivalent ventilation frequency $ NV $ for using ventilation in the heating and cooling section $ i = 2 to 5 $ is 5.0 times / h, When the use of ventilation corresponds to "use ventilation (20 times of ventilation / h equivalent or more)", the equivalent ventilation frequency $ NV $ in utilization of ventilation in heating and cooling section $ i = 2 to 5 $ let be 20.0 / h .

In [ ]:
# 標準住戸の負荷補正前の冷房顕熱負荷
def get_L_dash_CS_R_d_t_i(region, mode, NV_MR, NV_OR, Q, mu_C, i):
    if mode == 'い' or mode == '全館連続':
        # 通風の利用における相当換気回数ܸܰは、「住戸全体を連続的に冷房する方式」の場合はすべての暖冷房区画において0.0回/h とする。
        NV = 0.0
    elif mode in ['ろ', 'は'] or mode == '居室間歇':
        # 主たる居室
        if i == 1:
            if NV_MR == 0:
                NV = 0.0
            elif NV_MR == 5:
                NV = 5.0
            elif NV_MR == 20:
                NV = 20.0
            else:
                raise ValueError(NV_MR)
        # その他の居室
        elif 2 <= i and i <= 5:
            if NV_OR == 0:
                NV = 0.0
            elif NV_OR == 5:
                NV = 5.0
            elif NV_OR == 20:
                NV = 20.0
            else:
                raise ValueError(NV_OR)
        else:
            raise ValueError(i)
    else:
        raise ValueError(mode)

    NV1 = get_NV(1)
    NV2 = get_NV(2)
    NV3 = get_NV(3)

    # (11a)
    if NV == NV1:
        return get_L_dash_CS_R_NVl_d_t_i(region, mode, 1, Q, mu_C, i)
    elif NV == NV2:
        return get_L_dash_CS_R_NVl_d_t_i(region, mode, 2, Q, mu_C, i)
    elif NV == NV3:
        return get_L_dash_CS_R_NVl_d_t_i(region, mode, 3, Q, mu_C, i)
    elif NV < NV2:
        L_dash_CS_R_NV1_d_t_i = get_L_dash_CS_R_NVl_d_t_i(region, mode, 1, Q, mu_C, i)
        L_dash_CS_R_NV2_d_t_i = get_L_dash_CS_R_NVl_d_t_i(region, mode, 2, Q, mu_C, i)
        return (NV - NV2) / (NV1 - NV2) * L_dash_CS_R_NV1_d_t_i + (NV - NV1) / (NV2 - NV1) * L_dash_CS_R_NV2_d_t_i
    elif NV2 <= NV:
        L_dash_CS_R_NV2_d_t_i = get_L_dash_CS_R_NVl_d_t_i(region, mode, 2, Q, mu_C, i)
        L_dash_CS_R_NV3_d_t_i = get_L_dash_CS_R_NVl_d_t_i(region, mode, 3, Q, mu_C, i)
        return (NV - NV3) / (NV2 - NV3) * L_dash_CS_R_NV2_d_t_i + (NV - NV2) / (NV3 - NV2) * L_dash_CS_R_NV3_d_t_i
    else:
        raise NotImplementedError()


# 標準住戸の負荷補正前の冷房潜熱負荷
def get_L_dash_CL_R_d_t_i(region, mode, NV_MR, NV_OR, Q, mu_C, i):
    if mode == 'い' or mode == '全館連続':
        # 通風の利用における相当換気回数ܸܰは、「住戸全体を連続的に冷房する方式」の場合はすべての暖冷房区画において0.0回/h とする。
        NV = 0.0
    elif mode in ['ろ', 'は'] or mode == '居室間歇':
        # 主たる居室
        if i == 1:
            if NV_MR == 0:
                NV = 0.0
            elif NV_MR == 5:
                NV = 5.0
            elif NV_MR == 20:
                NV = 20.0
            else:
                raise ValueError(NV_MR)
        # その他の居室
        elif 2 <= i and i <= 5:
            if NV_OR == 0:
                NV = 0.0
            elif NV_OR == 5:
                NV = 5.0
            elif NV_OR == 20:
                NV = 20.0
            else:
                raise ValueError(NV_OR)
        else:
            raise ValueError(i)
    else:
        raise ValueError(mode)

    NV1 = get_NV(1)
    NV2 = get_NV(2)
    NV3 = get_NV(3)

    # (11b)
    if NV == NV1:
        return get_L_dash_CL_R_NVl_d_t_i(region, mode, 1, Q, mu_C, i)
    elif NV == NV2:
        return get_L_dash_CL_R_NVl_d_t_i(region, mode, 2, Q, mu_C, i)
    elif NV == NV3:
        return get_L_dash_CL_R_NVl_d_t_i(region, mode, 3, Q, mu_C, i)
    elif NV < NV2:
        L_dash_CL_R_NV1_d_t_i = get_L_dash_CL_R_NVl_d_t_i(region, mode, 1, Q, mu_C, i)
        L_dash_CL_R_NV2_d_t_i = get_L_dash_CL_R_NVl_d_t_i(region, mode, 2, Q, mu_C, i)
        return (NV - NV2) / (NV1 - NV2) * L_dash_CL_R_NV1_d_t_i + (NV - NV1) / (NV2 - NV1) * L_dash_CL_R_NV2_d_t_i
    elif NV2 <= NV:
        L_dash_CL_R_NV2_d_t_i = get_L_dash_CL_R_NVl_d_t_i(region, mode, 2, Q, mu_C, i)
        L_dash_CL_R_NV3_d_t_i = get_L_dash_CL_R_NVl_d_t_i(region, mode, 3, Q, mu_C, i)
        return (NV - NV3) / (NV2 - NV3) * L_dash_CL_R_NV2_d_t_i + (NV - NV2) / (NV3 - NV2) * L_dash_CL_R_NV3_d_t_i
    else:
        raise NotImplementedError()

<div style="text-align: center;font-weight: bold;">表　通風の利用に関する区分$l$の通風の利用における相当換気回数$NV_l$<br>Table: Use of ventilation Ventilation in the use of category $l$ of ventilation $ NV_l $</div>

|通風の利用に関する区分$l$<br>Classification $l$ about the use of ventilation|通風の利用における相当換気回数$NV_l$<br>Equivalent ventilation frequency in the use of ventilation $NV_l$|
|-|-|
|1|0.0|
|2|5.0|
|3|20.0|


In [ ]:
# 表 9 通風の利用に関する区分݈の通風の利用における相当換気回数 NV_l
def get_NV(l):
    table_9 = (0.0, 5.0, 20.0)
    return table_9[l - 1]

1～7 地域の場合

In the case of 1-7 areas

$$\large
L'_{CS,R,NV_l,d,t,i} = \left\{ \begin{array}{ll}
    \frac{Q - Q_2}{Q_1 - Q_2} \times L'_{CS,R,NV_l,Q_1,d,t,i} + \frac{Q - Q_1}{Q_2 - Q_1} \times L'_{CS,R,NV_l,Q_2,d,t,i} & (Q \ge Q_2の場合) \\
    \frac{Q - Q_3}{Q_2 - Q_3} \times L'_{CS,R,NV_l,Q_2,d,t,i} + \frac{Q - Q_2}{Q_3 - Q_2} \times L'_{CS,R,NV_l,Q_3,d,t,i} & (Q_2 > Q \ge Q_3の場合) \\
    \frac{Q - Q_4}{Q_3 - Q_4} \times L'_{CS,R,NV_l,Q_3,d,t,i} + \frac{Q - Q_3}{Q_4 - Q_3} \times L'_{CS,R,NV_l,Q_4,d,t,i} & (Q_3 > Qの場合) \\
\end{array}
\right.
$$

$$\large
L'_{CL,R,NV_l,d,t,i} = \left\{ \begin{array}{ll}
    \frac{Q - Q_2}{Q_1 - Q_2} \times L'_{CL,R,NV_l,Q_1,d,t,i} + \frac{Q - Q_1}{Q_2 - Q_1} \times L'_{CL,R,NV_l,Q_2,d,t,i} & (Q \ge Q_2の場合) \\
    \frac{Q - Q_3}{Q_2 - Q_3} \times L'_{CL,R,NV_l,Q_2,d,t,i} + \frac{Q - Q_2}{Q_3 - Q_2} \times L'_{CL,R,NV_l,Q_3,d,t,i} & (Q_2 > Q \ge Q_3の場合) \\
    \frac{Q - Q_4}{Q_3 - Q_4} \times L'_{CL,R,NV_l,Q_3,d,t,i} + \frac{Q - Q_3}{Q_4 - Q_3} \times L'_{CL,R,NV_l,Q_4,d,t,i} & (Q_3 > Qの場合) \\
\end{array}
\right.
$$

8 地域の場合

In the case of 8 area

$$\large
L'_{CS,R,NV_l,d,t,i} = \left\{ \begin{array}{ll}
    \frac{Q-Q_2}{Q_1 - Q_2} \times L'_{CS,R,NV_l,Q_1,d,t,i} + \frac{Q - Q_1}{Q_2 - Q_1} \times L'_{CS,R,NV_l,Q_2,d,t,i} & (Q \ge Q_2の場合) \\
    \frac{Q-Q_3}{Q_2 - Q_3} \times L'_{CS,R,NV_l,Q_2,d,t,i} + \frac{Q - Q_2}{Q_3 - Q_2} \times L'_{CS,R,NV_l,Q_3,d,t,i} & (Q_2 > Qの場合) \\
\end{array}
\right.
$$

$$\large
L'_{CL,R,NV_l,d,t,i} = \left\{ \begin{array}{ll}
    \frac{Q - Q_2}{Q_1 - Q_2} \times L'_{CL,R,NV_l,Q_1,d,t,i} + \frac{Q - Q_1}{Q_2 - Q_1} \times L'_{CL,R,NV_l,Q_2,d,t,i} & (Q \ge Q_2の場合) \\
    \frac{Q - Q_3}{Q_2 - Q_3} \times L'_{CL,R,NV_l,Q_2,d,t,i} + \frac{Q - Q_2}{Q_3 - Q_2} \times L'_{CL,R,NV_l,Q_3,d,t,i} & (Q_2 > Qの場合) \\
\end{array}
\right.
$$

$L'_{CS,R,NV_l,Q_j,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の通風の利用における相当換気回数$NV_l$の熱損失係数$Q_j$の標準住戸の負荷補正前の冷房顕熱負荷(MJ/h) / Cooling sensible heat load before load correction of standard dwelling unit with heat loss coefficient $ Q_j $ equivalent heat exchange factor $ NV_l $ in utilization of ventilation of heating / cooling section $ i $ at time $ t $ of date $ d $  
$L'_{CL,R,NV_l,Q_j,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の通風の利用における相当換気回数$NV_l$の熱損失係数$Q_j$の標準住戸の負荷補正前の冷房潜熱負荷(MJ/h) / Cooling latent heat load before load compensation of standard dwellings with the equivalent ventilation frequency $ NV_l $ heat loss coefficient $ Q_j $ in utilization of ventilation of heating / cooling compartment $ i $ at time $ t $ of date $ d $  

In [ ]:
# 冷房顕熱負荷のQ値按分 (12)
def get_L_dash_CS_R_NVl_d_t_i(region, mode, l, Q, mu_C, i):
    if region in [1, 2, 3, 4, 5, 6, 7]:

        Q1 = get_Q_j(region, 1)
        Q2 = get_Q_j(region, 2)
        Q3 = get_Q_j(region, 3)
        Q4 = get_Q_j(region, 4)

        # (12a)
        if Q == Q1:
            return get_L_dash_CS_R_NVl_Qj_d_t_i(region, mode, l, 1, mu_C, i)
        elif Q == Q2:
            return get_L_dash_CS_R_NVl_Qj_d_t_i(region, mode, l, 2, mu_C, i)
        elif Q == Q3:
            return get_L_dash_CS_R_NVl_Qj_d_t_i(region, mode, l, 3, mu_C, i)
        elif Q == Q4:
            return get_L_dash_CS_R_NVl_Qj_d_t_i(region, mode, l, 4, mu_C, i)
        elif Q >= Q2:
            L_dash_CS_R_NVl_Q1_d_t_i = get_L_dash_CS_R_NVl_Qj_d_t_i(region, mode, l, 1, mu_C, i)
            L_dash_CS_R_NVl_Q2_d_t_i = get_L_dash_CS_R_NVl_Qj_d_t_i(region, mode, l, 2, mu_C, i)
            return (Q - Q2) / (Q1 - Q2) * L_dash_CS_R_NVl_Q1_d_t_i + (Q - Q1) / (Q2 - Q1) * L_dash_CS_R_NVl_Q2_d_t_i
        elif Q2 > Q and Q >= Q3:
            L_dash_CS_R_NVl_Q2_d_t_i = get_L_dash_CS_R_NVl_Qj_d_t_i(region, mode, l, 2, mu_C, i)
            L_dash_CS_R_NVl_Q3_d_t_i = get_L_dash_CS_R_NVl_Qj_d_t_i(region, mode, l, 3, mu_C, i)
            return (Q - Q3) / (Q2 - Q3) * L_dash_CS_R_NVl_Q2_d_t_i + (Q - Q2) / (Q3 - Q2) * L_dash_CS_R_NVl_Q3_d_t_i
        elif Q3 > Q:
            L_dash_CS_R_NVl_Q3_d_t_i = get_L_dash_CS_R_NVl_Qj_d_t_i(region, mode, l, 3, mu_C, i)
            L_dash_CS_R_NVl_Q4_d_t_i = get_L_dash_CS_R_NVl_Qj_d_t_i(region, mode, l, 4, mu_C, i)
            return (Q - Q4) / (Q3 - Q4) * L_dash_CS_R_NVl_Q3_d_t_i + (Q - Q3) / (Q4 - Q3) * L_dash_CS_R_NVl_Q4_d_t_i
        else:
            raise NotImplementedError()
    elif region in [8]:

        Q1 = get_Q_j(region, 1)
        Q2 = get_Q_j(region, 2)
        Q3 = get_Q_j(region, 3)

        # (12c)
        if Q == Q1:
            return get_L_dash_CS_R_NVl_Qj_d_t_i(region, mode, l, 1, mu_C, i)
        elif Q == Q2:
            return get_L_dash_CS_R_NVl_Qj_d_t_i(region, mode, l, 2, mu_C, i)
        elif Q == Q3:
            return get_L_dash_CS_R_NVl_Qj_d_t_i(region, mode, l, 3, mu_C, i)
        elif Q >= Q2:
            L_dash_CS_R_NVl_Q1_d_t_i = get_L_dash_CS_R_NVl_Qj_d_t_i(region, mode, l, 1, mu_C, i)
            L_dash_CS_R_NVl_Q2_d_t_i = get_L_dash_CS_R_NVl_Qj_d_t_i(region, mode, l, 2, mu_C, i)
            return (Q - Q2) / (Q1 - Q2) * L_dash_CS_R_NVl_Q1_d_t_i + (Q - Q1) / (Q2 - Q1) * L_dash_CS_R_NVl_Q2_d_t_i
        elif Q2 > Q:
            L_dash_CS_R_NVl_Q2_d_t_i = get_L_dash_CS_R_NVl_Qj_d_t_i(region, mode, l, 2, mu_C, i)
            L_dash_CS_R_NVl_Q3_d_t_i = get_L_dash_CS_R_NVl_Qj_d_t_i(region, mode, l, 3, mu_C, i)
            return (Q - Q3) / (Q2 - Q3) * L_dash_CS_R_NVl_Q2_d_t_i + (Q - Q2) / (Q3 - Q2) * L_dash_CS_R_NVl_Q3_d_t_i
        else:
            raise ValueError(Q)
    else:
        raise ValueError(region)


# 冷房潜熱負荷のQ値按分 (12)
def get_L_dash_CL_R_NVl_d_t_i(region, mode, l, Q, mu_C, i):
    if region in [1, 2, 3, 4, 5, 6, 7]:

        Q1 = get_Q_j(region, 1)
        Q2 = get_Q_j(region, 2)
        Q3 = get_Q_j(region, 3)
        Q4 = get_Q_j(region, 4)

        # (12b)
        if Q == Q1:
            return get_L_dash_CL_R_NVl_Qj_d_t_i(region, mode, l, 1, mu_C, i)
        elif Q == Q2:
            return get_L_dash_CL_R_NVl_Qj_d_t_i(region, mode, l, 2, mu_C, i)
        elif Q == Q3:
            return get_L_dash_CL_R_NVl_Qj_d_t_i(region, mode, l, 3, mu_C, i)
        elif Q == Q4:
            return get_L_dash_CL_R_NVl_Qj_d_t_i(region, mode, l, 4, mu_C, i)
        elif Q >= Q2:
            L_dash_CL_R_NVl_Q1_d_t_i = get_L_dash_CL_R_NVl_Qj_d_t_i(region, mode, l, 1, mu_C, i)
            L_dash_CL_R_NVl_Q2_d_t_i = get_L_dash_CL_R_NVl_Qj_d_t_i(region, mode, l, 2, mu_C, i)
            return (Q - Q2) / (Q1 - Q2) * L_dash_CL_R_NVl_Q1_d_t_i + (Q - Q1) / (Q2 - Q1) * L_dash_CL_R_NVl_Q2_d_t_i
        elif Q2 > Q and Q >= Q3:
            L_dash_CL_R_NVl_Q2_d_t_i = get_L_dash_CL_R_NVl_Qj_d_t_i(region, mode, l, 2, mu_C, i)
            L_dash_CL_R_NVl_Q3_d_t_i = get_L_dash_CL_R_NVl_Qj_d_t_i(region, mode, l, 3, mu_C, i)
            return (Q - Q3) / (Q2 - Q3) * L_dash_CL_R_NVl_Q2_d_t_i + (Q - Q2) / (Q3 - Q2) * L_dash_CL_R_NVl_Q3_d_t_i
        elif Q3 > Q:
            L_dash_CL_R_NVl_Q3_d_t_i = get_L_dash_CL_R_NVl_Qj_d_t_i(region, mode, l, 3, mu_C, i)
            L_dash_CL_R_NVl_Q4_d_t_i = get_L_dash_CL_R_NVl_Qj_d_t_i(region, mode, l, 4, mu_C, i)
            return (Q - Q4) / (Q3 - Q4) * L_dash_CL_R_NVl_Q3_d_t_i + (Q - Q3) / (Q4 - Q3) * L_dash_CL_R_NVl_Q4_d_t_i
        else:
            raise ValueError(Q)
    elif region in [8]:

        Q1 = get_Q_j(region, 1)
        Q2 = get_Q_j(region, 2)
        Q3 = get_Q_j(region, 3)

        # (12d)
        if Q == Q1:
            return get_L_dash_CL_R_NVl_Qj_d_t_i(region, mode, l, 1, mu_C, i)
        elif Q == Q2:
            return get_L_dash_CL_R_NVl_Qj_d_t_i(region, mode, l, 2, mu_C, i)
        elif Q == Q3:
            return get_L_dash_CL_R_NVl_Qj_d_t_i(region, mode, l, 3, mu_C, i)
        elif Q >= Q2:
            L_dash_CL_R_NVl_Q1_d_t_i = get_L_dash_CL_R_NVl_Qj_d_t_i(region, mode, l, 1, mu_C, i)
            L_dash_CL_R_NVl_Q2_d_t_i = get_L_dash_CL_R_NVl_Qj_d_t_i(region, mode, l, 2, mu_C, i)
            return (Q - Q2) / (Q1 - Q2) * L_dash_CL_R_NVl_Q1_d_t_i + (Q - Q1) / (Q2 - Q1) * L_dash_CL_R_NVl_Q2_d_t_i
        elif Q2 > Q:
            L_dash_CL_R_NVl_Q2_d_t_i = get_L_dash_CL_R_NVl_Qj_d_t_i(region, mode, l, 2, mu_C, i)
            L_dash_CL_R_NVl_Q3_d_t_i = get_L_dash_CL_R_NVl_Qj_d_t_i(region, mode, l, 3, mu_C, i)
            return (Q - Q3) / (Q2 - Q3) * L_dash_CL_R_NVl_Q2_d_t_i + (Q - Q2) / (Q3 - Q2) * L_dash_CL_R_NVl_Q3_d_t_i
        else:
            raise ValueError(Q)
    else:
        raise ValueError(region)

$$\large
L'_{CS,R,NV_l,Q_j,d,t,i} = \left\{ \begin{array}{ll}
    \frac{\mu_C - \mu_{C,j,2}}{\mu_{C,j,1} - \mu_{C,j,2}} \times L'_{CS,R,NV_l,Q_j,\mu_{C,j,1},d,t,i} + \frac{\mu_C - \mu_{C,j,1}}{\mu_{C,j,2} - \mu_{C,j,1}} \times L'_{CS,R,NV_l,Q_j,\mu_{C,j,2},d,t,i} & (\mu_C < \mu_{C,j,2}) \\
    \frac{\mu_C - \mu_{C,j,3}}{\mu_{C,j,2} - \mu_{C,j,3}} \times L'_{CS,R,NV_l,Q_j,\mu_{C,j,2},d,t,i} + \frac{\mu_C - \mu_{C,j,2}}{\mu_{C,j,3} - \mu_{C,j,2}} \times L'_{CS,R,NV_l,Q_j,\mu_{C,j,3},d,t,i} & (\mu_{C,j,2} \le \mu_C) \\
\end{array}
\right.
$$

$$\large
L'_{CL,R,NV_l,Q_j,d,t,i} = \left\{ \begin{array}{ll}
    \frac{\mu_C - \mu_{C,j,2}}{\mu_{C,j,1} - \mu_{C,j,2}} \times L'_{CL,R,NV_l,Q_j,\mu_{C,j,1},d,t,i} + \frac{\mu_C - \mu_{C,j,1}}{\mu_{C,j,2} - \mu_{C,j,1}} \times L'_{CL,R,NV_l,Q_j,\mu_{C,j,2},d,t,i} & (\mu_C < \mu_{C,j,2}) \\
    \frac{\mu_C - \mu_{C,j,3}}{\mu_{C,j,2} - \mu_{C,j,3}} \times L'_{CL,R,NV_l,Q_j,\mu_{C,j,2},d,t,i} + \frac{\mu_C - \mu_{C,j,2}}{\mu_{C,j,3} - \mu_{C,j,2}} \times L'_{CL,R,NV_l,Q_j,\mu_{C,j,3},d,t,i} & (\mu_{C,j,2} \le \mu_C) \\
\end{array}
\right.
$$


$L'_{CS,R,NV_l,Q_j,\mu_{C,j,k},d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の通風の利用における相当換気回数$NV_l$の熱損失係数$Q_j$かつ冷房期の日射取得係数$\mu_{C,j,k}$の標準住戸の負荷補正前の冷房顕熱負荷(MJ/h) / Heat loss coefficient $ Q_j $ of equivalent ventilation frequency $ NV_l $ in utilization of ventilation of heating / cooling section $ i $ at time $ t $ of date $ d $ and solar radiation acquisition  coefficient $\mu_{C,j,k},d,t,i$ during cooling period Cooling sensible heat load before load compensation for standard dwelling unit  
$L'_{CL,R,NV_l,Q_j,\mu_{C,j,1},d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の通風の利用における相当換気回数$NV_l$の熱損失係数$Q_j$かつ冷房期の日射取得係数$\mu_{C,j,k}$の標準住戸の負荷補正前の冷房潜熱負荷(MJ/h) / Heat loss coefficient $ Q_j $ of equivalent ventilation frequency $ NV_l $ in utilization of ventilation of heating / cooling section $ i $ at time $ t $ of date $ d $ and solar radiation acquisition coefficient $\mu_{C,j,k},d,t,i$ during cooling period Cooling latent heat load before load compensation for  standard dwelling unit   
$\mu_C$：冷房期の日射取得係数 / Solar radiation acquisition coefficient during the cooling period   
$\mu_{C,j,k}$：断熱性能の区分$j$における日射取得性能の区分$k$の冷房期の日射取得係数 / Division $ j $ of insulation performance division Division of solar radiation acquisition performance division Solar radiation acquisition coefficient of cooling period during $ k $  

In [ ]:
# 冷房顕熱負荷のμ値按分 (13a)
def get_L_dash_CS_R_NVl_Qj_d_t_i(region, mode, l, j, mu_C, i):
    mu_C_j_1 = get_mu_C_j_k(region, j, 1)
    mu_C_j_2 = get_mu_C_j_k(region, j, 2)
    mu_C_j_3 = get_mu_C_j_k(region, j, 3)

    if mu_C == mu_C_j_1:
        return get_L_dash_CS_R_NVl_Qj_muH_j_k_d_t_i(region, mode, l, j, 1, i)
    elif mu_C == mu_C_j_2:
        return get_L_dash_CS_R_NVl_Qj_muH_j_k_d_t_i(region, mode, l, j, 2, i)
    elif mu_C == mu_C_j_3:
        return get_L_dash_CS_R_NVl_Qj_muH_j_k_d_t_i(region, mode, l, j, 3, i)
    elif mu_C < mu_C_j_2:
        L_dash_CS_R_NVl_Qj_muH_j_1_d_t_i = get_L_dash_CS_R_NVl_Qj_muH_j_k_d_t_i(region, mode, l, j, 1, i)
        L_dash_CS_R_NVl_Qj_muH_j_2_d_t_i = get_L_dash_CS_R_NVl_Qj_muH_j_k_d_t_i(region, mode, l, j, 2, i)
        return (mu_C - mu_C_j_2) / (mu_C_j_1 - mu_C_j_2) * L_dash_CS_R_NVl_Qj_muH_j_1_d_t_i \
               + (mu_C - mu_C_j_1) / (mu_C_j_2 - mu_C_j_1) * L_dash_CS_R_NVl_Qj_muH_j_2_d_t_i
    elif mu_C_j_2 <= mu_C:
        L_dash_CS_R_NVl_Qj_muH_j_2_d_t_i = get_L_dash_CS_R_NVl_Qj_muH_j_k_d_t_i(region, mode, l, j, 2, i)
        L_dash_CS_R_NVl_Qj_muH_j_3_d_t_i = get_L_dash_CS_R_NVl_Qj_muH_j_k_d_t_i(region, mode, l, j, 3, i)
        return (mu_C - mu_C_j_3) / (mu_C_j_2 - mu_C_j_3) * L_dash_CS_R_NVl_Qj_muH_j_2_d_t_i \
               + (mu_C - mu_C_j_2) / (mu_C_j_3 - mu_C_j_2) * L_dash_CS_R_NVl_Qj_muH_j_3_d_t_i
    else:
        raise ValueError(mu_C)


# 冷房潜熱負荷のμ値按分 (13b)
def get_L_dash_CL_R_NVl_Qj_d_t_i(region, mode, l, j, mu_C, i):
    mu_C_j_1 = get_mu_C_j_k(region, j, 1)
    mu_C_j_2 = get_mu_C_j_k(region, j, 2)
    mu_C_j_3 = get_mu_C_j_k(region, j, 3)

    if mu_C == mu_C_j_1:
        return get_L_dash_CL_R_NVl_Qj_muH_j_k_d_t_i(region, mode, l, j, 1, i)
    elif mu_C == mu_C_j_2:
        return get_L_dash_CL_R_NVl_Qj_muH_j_k_d_t_i(region, mode, l, j, 2, i)
    elif mu_C == mu_C_j_3:
        return get_L_dash_CL_R_NVl_Qj_muH_j_k_d_t_i(region, mode, l, j, 3, i)
    elif mu_C < mu_C_j_2:
        L_dash_CL_R_NVl_Qj_muH_j_1_d_t_i = get_L_dash_CL_R_NVl_Qj_muH_j_k_d_t_i(region, mode, l, j, 1, i)
        L_dash_CL_R_NVl_Qj_muH_j_2_d_t_i = get_L_dash_CL_R_NVl_Qj_muH_j_k_d_t_i(region, mode, l, j, 2, i)
        return (mu_C - mu_C_j_2) / (mu_C_j_1 - mu_C_j_2) * L_dash_CL_R_NVl_Qj_muH_j_1_d_t_i \
               + (mu_C - mu_C_j_1) / (mu_C_j_2 - mu_C_j_1) * L_dash_CL_R_NVl_Qj_muH_j_2_d_t_i
    elif mu_C_j_2 <= mu_C:
        L_dash_CL_R_NVl_Qj_muH_j_2_d_t_i = get_L_dash_CL_R_NVl_Qj_muH_j_k_d_t_i(region, mode, l, j, 2, i)
        L_dash_CL_R_NVl_Qj_muH_j_3_d_t_i = get_L_dash_CL_R_NVl_Qj_muH_j_k_d_t_i(region, mode, l, j, 3, i)
        return (mu_C - mu_C_j_3) / (mu_C_j_2 - mu_C_j_3) * L_dash_CL_R_NVl_Qj_muH_j_2_d_t_i \
               + (mu_C - mu_C_j_2) / (mu_C_j_3 - mu_C_j_2) * L_dash_CL_R_NVl_Qj_muH_j_3_d_t_i
    else:
        raise ValueError(mu_C)

### 8．熱損失係数の計算方法 / Calculation method of heat loss coefficient

地域の区分が 8 地域又は熱交換型換気設備を採用していない場合は、熱損失係数$Q$に等しいとする。 

If the division of the area does not adopt 8 areas or heat exchange type ventilation equipment, it shall be equal to the heat loss coefficient $ Q $.

$$\large
Q_{HEXC} = Q - C_{V} \times r_{V} \times \eta'_{t}
$$

$Q$：熱損失係数(W/m<sup>2</sup>K) / Heat loss coefficient  
$C_{V}$：空気の容積比熱(Wh/m<sup>2</sup>K) / Volumetric specific heat of air  
$r_{V}$：床面積当たりの換気量の比((m<sup>3</sup>/h)/m<sup>2</sup>) / Ratio of ventilation volume per floor area  
$\eta'_{t}$：熱交換型換気設備の補正温度交換効率 / Corrected temperature exchange efficiency of heat exchange type ventilation equipment  

In [ ]:
# 熱交換型換気設備による暖房負荷低減を考慮した補正熱損失係数
# etr_dash_t: 熱交換型換気設備の補正温度交換効率[-]
def get_Q_HEXC(region, Q, hex, etr_dash_t):
    # 地域の区分が 1 地域～７地域において熱交換型換気設備を採用している場合は、式(14)により表されることとし、
    # 地域の区分が 8 地域又は熱交換型換気設備を採用していない場合は、熱損失係数ܳに等しいとする。
    if region == 8 or hex == False:
        return Q
    elif region in [1, 2, 3, 4, 5, 6, 7]:
        C_V = 0.35  # 空気の容積比熱
        r_V = 4.0 / 3.0  # 床面積当たりの換気量の比
        return Q - C_V * r_V * etr_dash_t  # (14)
    else:
        raise ValueError((region, hex))


空気の容積比熱$C_{V}$は0.35とし、床面積当たりの換気量の比$r_{V}$は4⁄3（床面積120m<sup>2</sup> に対する換気量160m<sup>3</sup>/h）とする。

The volume specific heat of air $ C_ {V} $ is 0.35, and the ratio of ventilation volume per floor area $ r_ {V} $ is 4⁄3 (ventilation volume 160 m<sup>3</sup> / h for floor area 120 m<sup>2</sup>).

$$\large
Q = Q' + 0.35 \times 0.5 \times 2.4
$$

$Q'$：熱損失係数数（換気による熱損失を含まない）(W/m<sup>2</sup>K) / Heat loss factor number (does not include heat loss due to ventilation)  

In [ ]:
# 熱損失係数
# Q_dash: ：熱損失係数（換気による熱損失を含まない）(W/m2K)
def get_Q(Q_dash):
    return Q_dash + 0.35 * 0.5 * 2.4  # (15)

### 9．暖冷房区画$i$の床面積 / Floor area of heating / cooling section $ i $

$$\large
A_{HCZ,i} = \left\{ \begin{array}{ll}
   A_{HCZ,R,i} \times \frac{A_{MR}}{A_{MR,R}} & (i = 1) \\
   A_{HCZ,R,i} \times \frac{A_{OR}}{A_{OR,R}} & (i = 2～5) \\
   A_{HCZ,R,i} \times \frac{(A_{A}-A_{MR}-A_{OR})}{A_{NO,R}} & (i = 6～12) \\
\end{array}
\right.
$$

$A_{HCZ,R,i}$：標準住戸における暖冷房区画$i$の床面積(m<sup>2</sup>) / Floor area of heating / cooling section $ i $ in standard dwelling unit  
$A_{MR}$：主たる居室の床面積(m<sup>2</sup>) / Floor area of main room  
$A_{OR}$：その他の居室の床面積(m<sup>2</sup>) / Floor area of other rooms  
$A_{A}$：床面積の合計(m<sup>2</sup>) / Total floor area  
$A_{MR,R}$：標準住戸の主たる居室の床面積(m<sup>2</sup>) / Floor area of main room of standard dwelling unit  
$A_{OR,R}$：標準住戸のその他の居室の床面積(m<sup>2</sup>) / Floor area of other rooms in standard units  
$A_{NO,R}$：標準住戸の非居室の床面積(m<sup>2</sup>) / Floor area of non-living room of standard dwelling unit  

In [ ]:
# 暖冷房区画iの床面積 (16)
def get_A_HCZ_i(i, A_A, A_MR, A_OR):
    """
    :param i: 暖冷房区画の番号
    :param A_A: 床面積の合計 (m2)
    :param A_MR: 主たる居室の床面積 (m2)
    :param A_OR: その他の居室の床面積 (m2)
    :return: 暖冷房区画iの床面積 (m2)
    """
    # 標準住戸における暖冷房区画iの床面積
    A_HCZ_R_i = get_A_HCZ_R_i(i)

    A_MR_R = 29.81
    A_OR_R = 51.34
    A_NO_R = 38.93

    # 暖冷房区画iの床面積を按分して求める
    if i == 1:
        return A_HCZ_R_i * A_MR / A_MR_R
    elif 2 <= i and i <= 5:
        return A_HCZ_R_i * A_OR / A_OR_R
    elif 6 <= i and i <= 12:
        return A_HCZ_R_i * (A_A - A_MR - A_OR) / A_NO_R
    else:
        raise ValueError(i)

<div style="text-align: center;font-weight: bold;">表　標準住戸における主たる居室、その他の居室及び非居室の面積、並びに暖冷房区画݅の床面積<br>Table: Main living room in standard dwelling unit, area of other living rooms and non-living rooms, and floor area of heating and cooling compartments</div>

<table>
<thead>
<tr>
<th>暖冷房区画$i$の番号<br>Number of heating / cooling section $ i $</th>
<th>（参考）<br>想定する居室の種類<br>(Reference) <br>Type of room to be assumed</th>
<th>居室の種類<br>Type of room</th>
<th>暖冷房区画$i$の<br>床面積(m<sup>2</sup>)<br>Floor area of heating / cooling section $ i $</th>
<th>主たる居室、<br>その他の居室、<br>非居室の床面積<br>(m<sup>2</sup>)<br>Main living room, <br>other living room, <br>floor space of non-living room</th>
</tr>
</thead>
    
<tbody>
<tr>
<th>1</th>
<th>居間食堂（LD）・台所（K）<br>Living room dining room (LD) / kitchen (K)</th>
<th>主たる居室<br>Main room</th>
<td>29.81</td>
<td>29.81</td>
</tr>
<tr>
<th>2</th>
<th>和室<br>Japanese room</th>
<th rowspan="4">その他の居室<br>Other rooms</th>
<td>16.56</td>
<td rowspan="4">51.34</td>
</tr>
<tr>
<th>3</th>
<th>主寝室<br>Master bedroom</th>
<td>13.25</td>
</tr>
<tr>
<th>4</th>
<th>子供室 1<br>Children's room 1</th>
<td>10.76</td>
</tr>
<tr>
<th>5</th>
<th>子供室 2<br>Children's room 2</th>
<td>10.77</td>
</tr>
<tr>
<th>6</th>
<th>浴室<br>bathroom</th>
<th rowspan="7">非居室<br>No living room</th>
<td>3.31</td>
<td rowspan="7">38.93
</td>
</tr>
<tr>
<th>7</th>
<th>1F 便所<br>1F Toilet</th>
<td>1.66</td>
</tr>
<tr>
<th>8</th>
<th>洗面所<br>Washroom</th>
<td>3.31</td>
</tr>
<tr>
<th>9</th>
<th>ホール<br>hall</th>
<td>13.25</td>
</tr>
<tr>
<th>10</th>
<th>クローゼット<br>closet</th>
<td>4.97</td>
</tr>
<tr>
<th>11</th>
<th>2F ホール<br>2F hall</th>
<td>10.77</td>
</tr>
<tr>
<th>12</th>
<th>2F 便所<br>2F Toilet</th>
<td>1.66</td>
</tr>
</tbody>
</table>

In [ ]:
# 標準住戸における暖冷房区画iの床面積
def get_A_HCZ_R_i(i):
    # 表10 標準住戸における主たる居室、その他の居室及び非居室の面積、並びに暖冷房区画iの床面積
    table_10 = [
        29.81,
        16.56,
        13.25,
        10.76,
        10.77,
        3.31,
        1.66,
        3.31,
        13.25,
        4.97,
        10.77,
        1.66
    ]
    return table_10[i - 1]


### 10．暖房日 / Heating day

暖房日は暖房使用が発生することが見込まれる日と定義し、当該日の前日において標準住戸全体の負荷補正前の暖房負荷を日積算した値がゼロを超える場合（すなわち、$0 < \sum_{t=0}^{23} \sum_{i=1}^{12} L'_{H,R,d-1,t,i}$ の場合）、日付$d$は暖房日とする。ただし、日積算においては、日付$d-1$の時刻$t$における暖冷房区画$i$の標準住戸の負荷補正前の暖房負荷$L'_{H,R,d-1,t,i}$は、ゼロ未満の場合はゼロとする。また、日付$d$が 1 月 1 日である場合、日付݀は暖房日とする。 

A heating day is defined as a day when heating use is expected to occur, and if the value obtained by integrating the heating load before the load correction of the entire standard dwellings on the day before that day exceeds zero (That is, in the case of $ 0 <\sum_ {t = 0} ^ {23} \sum_ {i = 1} ^ {12} L '_ {H, R, d-1, t, i} $), the date $ d $ Is a heating day. However, in the daily integration, the heating load $L'_{H,R,d-1,t,i}$ before the load correction of the standard dwelling unit $ i $ of the heating / cooling section $ i $ at the time $ t $ of the date $ d-1 $ is zero if less than zero. Also, if the date $ d $ is January 1, the date is the heating day.

In [ ]:
# 暖房日
# 暖房日 = 暖房使用が発生することが見込まれる日
def get_heating_flag_d(L_dash_H_R_d_t_i):
    L_dash_H_R_d_t = np.sum(L_dash_H_R_d_t_i, axis=0)
    L_dash_H_R_d_t = np.clip(L_dash_H_R_d_t, 0, None)
    L_dash_H_R_d = np.sum(L_dash_H_R_d_t.reshape(365, 24), axis=1)

    heating_flag_d = np.ones(365)
    heating_flag_d[1:] = L_dash_H_R_d[0:364] > 0

    return heating_flag_d

## 付録 A 熱交換型換気設備 / Appendix A Heat exchange ventilation system

### A.1 記号及び単位 / Symbol and unit

省略

Omission

### A.2 熱交換型換気設備の補正温度交換効率 / Corrected temperature exchange efficiency of heat exchange type ventilation equipment

$$\large
\eta'_{t} = \eta_{t} \times C_{tol} \times C_{eff} \times C_{bal} \times C_{leak}
$$

$\eta'_{t}$：熱交換型換気設備の補正温度交換効率 / Corrected temperature exchange efficiency of heat exchange type ventilation equipment  
$\eta_{t}$：熱交換型換気設備の温度交換効率 / Temperature exchange efficiency of heat exchange type ventilation equipment  
$C_{tol}$：カタログ表記誤差による温度交換効率の補正係数 / Correction factor of temperature exchange efficiency by catalog notation error  
$C_{eff}$：有効換気量率による温度交換効率の補正係数 / Correction factor of temperature exchange efficiency by effective ventilation rate rate  
$C_{bal}$：給気と排気の比率による温度交換効率の補正係数 / Correction factor of temperature exchange efficiency by ratio of air supply and exhaust  
$C_{leak}$：排気過多時における住宅外皮経由の漏気による温度交換効率の補正係数 / Correction factor of temperature exchange efficiency due to air leakage through the housing skin at the time of exhaust excess  

### A.3 熱交換型換気設備の温度交換効率 / Temperature exchange efficiency of heat exchange type ventilation equipment

温度交換効率$\eta_{t}$が0.4を下回る場合、又は、定格条件における給気風量が定格条件における還気風量の半分未満、若しくは 2 倍より大きい場合は、熱交換型換気設備による暖房負荷の削減効果を見込むことはできない。また、温度交換効率$\eta_{t}$が0.95を上回る場合は、温度交換効率$\eta_{t}$を0.95とする。 

If the temperature exchange efficiency $ \eta_ {t} $ is less than 0.4, or if the supply air volume at rated conditions is less than half or more than twice the return air volume at rated conditions, heating by heat exchange ventilators We can not anticipate the reduction effect of the load. If the temperature exchange efficiency $ \eta_ {t} $ exceeds 0.95, the temperature exchange efficiency $ \eta_ {t} $ is set to 0.95.

### A.4 カタログ表記誤差による温度交換効率の補正係数 / Correction factor of temperature exchange efficiency by catalog notation error

カタログ表記誤差による温度交換効率の補正係数$C_{tol}$は、0.95とする。 

The correction coefficient $ C_ {tol} $ of the temperature exchange efficiency due to the catalog notation error is 0.95.

### A.5 有効換気量率による温度交換効率の補正係数 / Correction factor of temperature exchange efficiency by effective ventilation rate rate

有効換気量率による温度交換効率の補正係数$C_{eff}$が 0 未満の値となる場合は、有効換気量率による温度交換効率の補正係数$C_{eff}$は 0 に等しいとする。

If the correction coefficient $ C_ {eff} $ of the temperature exchange efficiency by the effective ventilation rate ratio is less than 0, the correction coefficient $ C_ {eff} $ of the temperature exchange efficiency by the effective ventilation volume rate is equal to 0.

$$\large
C_{eff} = 1 - \frac{(\frac{1}{e}-1)(1-\eta_{t})}{\eta_{t}}
$$

$e$：全般換気設備の有効換気量率 / Effective ventilation rate rate of general ventilation equipment  
$\eta_{t}$：熱交換型換気設備の温度交換効率 / Temperature exchange efficiency of heat exchange type ventilation equipment  

### A.6 給気と排気の比率による温度交換効率の補正係数 / Correction factor of temperature exchange efficiency by ratio of air supply and exhaust

給気と排気の比率による温度交換効率の補正係数$C_{bal}$は0.90を用いるか、以下に示す方法に依ることができる。

The correction coefficient $ C_ {bal} $ of the temperature exchange efficiency according to the ratio of air supply to exhaust can use 0.90 or can be according to the method described below.

$$\large
C_{bal} = \frac{\eta_{t,d}}{\eta_{t}}
$$

$\eta_{t,d}$：当該住戸における補正設計風量比での熱交換型換気設備の温度交換効率 / Temperature exchange efficiency of heat exchange type ventilation equipment at corrected design air volume ratio in the dwelling unit concerned  
$\eta_{t}$：熱交換型換気設備の温度交換効率 / Temperature exchange efficiency of heat exchange type ventilation equipment  

$$\large
\eta_{t,d} = \left\{ \begin{array}{ll}
   \eta_{d} & (V_{d,RA} > V_{d,SA}) \\
   \eta_{d} \times R'_{vnt,d} & (V_{d,RA} \leq V_{d,SA}) \\
\end{array}
\right.
$$

$\eta_{d}$：当該住戸における補正設計風量比での熱通過有効度 / Heat passing effectiveness at the corrected design air volume ratio in the relevant dwelling unit  
$R'_{vnt,d}$：当該住戸における補正設計風量比 / Corrected design air volume ratio in the relevant dwelling unit  
$V_{d,SA}$：当該住戸における設計給気風量(m<sup>3</sup>/h) / Design supply air volume in the relevant dwelling unit  
$V_{d,RA}$：当該住戸における設計還気風量(m<sup>3</sup>/h) / Design return air volume at the relevant unit  

当該住戸における補正設計風量比での熱通過有効度$\eta_{d}$は、式により表される。

The heat passing effectiveness factor $ \ eta_ {d} $ at the corrected design air volume ratio in the relevant dwelling unit is expressed by the equation.

直交流型熱交換器の場合

In the case of a cross flow heat exchanger

$$\large
\eta_{d} = 1-e^{ \Biggl[ \frac{e^{\big(-N_{d}^{0.78}・R'_{vnt,d}\big)}-1}{ N_{d}^{-0.22}・R'_{vnt,d} } \Biggr]}
$$

向流-直交流複合型熱交換器の場合

In the case of a countercurrent-crossflow combined heat exchanger

$$\large
\eta_{d} = \frac{ 1-e^{ \Biggl[ -(1-R'_{vnt,d}) \Big( 1+ \frac{\frac{b}{l} sin\ \alpha\ cos\ \alpha}{0.0457143N_{d}^{2}+0.0691429N_{d}+0.9954286}  \Big) N_{d} \Biggr]} }{ 1-R'_{vnt,d}・e^{ \Biggl[ -(1-R'_{vnt,d}) \Big( 1+ \frac{\frac{b}{l} sin\ \alpha\ cos\ \alpha}{0.0457143N_{d}^{2}+0.0691429N_{d}+0.9954286}  \Big) N_{d} \Biggr]} }
$$

$N_{d}$：当該住戸における設計風量比での伝熱単位数 / Number of heat transfer units at the design air volume ratio in the relevant dwelling unit  
$b$：向流-直交流複合型熱交換器の向流部の幅(m) / Countercurrent width of countercurrent-crossflow combined heat exchanger  
$l$：向流-直交流複合型熱交換器の向流部の長さ(m) / Length of counterflow part of countercurrent-crossflow combined heat exchanger  
$\alpha$：向流-直交流複合型熱交換器の向流部と直交流部の接続角度(°) / Connection angle between the counterflow and crossflow of the countercurrent-crossflow combined heat exchanger  

ただし、向流-直交流複合型熱交換器の向流部の幅$b$が0.01(m)以上2.0(m)以下、かつ向流部の長さ$l$が0.01(m)以上3.0(m)以下、かつ向流部と直交流部の接続角度$\alpha$が15°以上60°以下を満たしていない場合は熱交換型換気設備による暖房負荷の削減効果を見込むことはできない。 

However, the width $ b $ of the counterflow part of the countercurrent-crossflow combined heat exchanger is 0.01 (m) or more and 2.0 (m) or less, and the length $ l $ of the counterflow part is 0.01 (m) or more and 3.0 (m) If the connection angle $ \alpha $ between the countercurrent part and the crossflow part does not satisfy 15 ° or more and 60 ° or less, it is impossible to anticipate the heating load reduction effect of the heat exchange ventilator.

In [1]:
# 熱交換型換気設備の補正熱交換効率 (-) (1)
def calc_etr_dash_t(etr_t_raw, e, C_bal, C_leak):
    """
    :param etr_t_raw: 熱交換型換気設備の温度交換効率 (-)
    :param e: 有効換気量率 (-)
    :param C_bal: 給気と排気の比率による温度交換効率の補正係数 (-)
    :param C_leak: 排気過多時における住宅外皮経由の漏気による温度交換効率の補正係数 (-)
    :return: 熱交換型換気設備の補正熱交換効率 (-)
    """
    # 熱交換型換気設備の温度交換効率
    etr_t = get_etr_t(etr_t_raw)

    # カタログ表記誤差による温度交換効率の補正係数 (-)
    C_tol = get_C_tol()

    # 有効換気量率による温度交換効率の補正係数 (2)
    C_eff = get_C_eff(etr_t, e)

    # 熱交換型換気設備の補正熱交換効率 (-) (1)
    etr_dash_t = get_etr_dash_t(etr_t, C_tol, C_eff, C_bal, C_leak)

    return etr_dash_t

In [ ]:
# 熱交換型換気設備の補正熱交換効率 (-) (1)
def get_etr_dash_t(etr_t, C_tol, C_eff, C_bal, C_leak):
    """
    :param etr_t: 熱交換型換気設備の温度交換効率 (-)
    :param C_tol: カタログ表示誤差による温度交換効率の補正係数 (-)
    :param C_eff: 有効換気量率による温度交換効率の補正係数 (-)
    :param C_bal: 給気と排気の比率による温度交換効率の補正係数 (-)
    :param C_leak: 排気過多時における住宅外皮経由の漏気による温度交換効率の補正係数 (-)
    :return: 熱交換型換気設備の補正熱交換効率 (-)
    """
    etr_dash_t = etr_t * C_tol * C_eff * C_bal * C_leak  # (1)
    return etr_dash_t

In [ ]:
# 熱交換型換気設備の温度交換効率
def get_etr_t(etr_t_raw):
    # 温度交換効率の値は、100 分の 1 未満の端数を切り下げた小数第二位までの値
    etr_t = ceil(etr_t_raw * 100) / 100
    if etr_t > 0.95:
        return 0.95
    else:
        return etr_t

In [ ]:
# カタログ表記誤差による温度交換効率の補正係数
def get_C_tol():
    return 0.95

In [ ]:
# 有効換気量率による温度交換効率の補正係数 (2)
def get_C_eff(etr_t, e):
    """
    :param etr_t: 熱交換型換気設備の温度交換効率 (-)
    :param e: 有効換気量率 (-)
    :return: 有効換気量率による温度交換効率の補正係数 (-)
    """
    C_eff = 1 - ((1 / e - 1) * (1 - etr_t) / etr_t)  # (2)

    #100 分の 1 未満の端数を切り下げた小数第二位までの値
    C_eff = floor(C_eff * 100) / 100
    if C_eff < 0.0:
        C_eff = 0.0

    return C_eff

In [ ]:
C_bal_default = 0.90


def get_C_bal():
    return etr_t_d / etr_t_d  # (3)


def get_etr_t_d():
    # (4)
    if V_d_RA > V_d_SA:
        return etr_d
    elif V_d_RA <= V_d_SA:
        return etr_d * R_dash_vnt_d


def get_etr_d_ac():
    # (5a)
    return 1 - exp((exp(-pow(N_d, 0.78) * R_dash_vnt_d) - 1) / (pow(N_d, -0.22) * R_dash_vnt_d))


def get_etr_d_dc():
    return (1 - exp(-(1 - R_dash_vnt_d) * (
            1 + ((b / l) * sin(alpha) * cos(alpha) / (0.0457143 * N_d ** 2 + 0.0691429 * N_d + 0.9954286))) * N_d)) \
           / (1 - R_dash_vnt_d * exp(-(1 - R_dash_vnt_d) * (
            ((b / l) * sin(alpha) * cos(alpha)) / (0.00457143 * N_d ** 2 + 0.0691429 ** N_d + 0.9954286)) * N_d))


## 付録 B 蓄熱の利用 / Appendix B Use of heat storage

省略

Omission

## 付録 C 通風を確保する措置の有無の判定 / Appendix C Judgment of existence of measures to secure ventilation

省略

Omission